# Laboratorul 5 

## Introducere
Acest document prezintă o aplicație complexă dezvoltată în Python, concepută pentru procesarea și manipularea imaginilor folosind biblioteci moderne de grafică și editare vizuală. Aplicația oferă o interfață grafică modernă și intuitivă pentru manipularea imaginilor cu următoarele caracteristici principale:

1. Suport pentru diverse spații de culoare (RGB, Grayscale, HSV etc.)
2. Gestionarea și conversia între multiple formate de imagine (BMP, JPEG, PNG etc.)
3. Implementarea unor tehnici de editare a imaginilor, precum ajustarea luminozității, contrastului și aplicarea filtrelor
4. Instrumente pentru scalare și rotație eficientă a imaginilor
5. Interfață grafică prietenoasă și ușor de utilizat pentru manipularea diverselor funcționalități

Proiectul demonstrează aplicarea practică a conceptelor fundamentale de procesare a imaginilor, oferind o platformă utilă pentru explorarea tehnicilor moderne de editare și analiză vizuală.

## Instalarea modulelor necesare
Din pricina faptului că sunt folosite anumite module care nu sunt prevăzute în mod implicit în Python Vanilla, va trebui să le importăm, lucru care se poate realiza prin rularea codului de mai jos

In [2]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QPushButton, 
                            QVBoxLayout, QHBoxLayout, QWidget, QFileDialog, 
                            QComboBox, QSlider, QTabWidget, QGroupBox, QGridLayout,
                            QScrollArea, QSplitter, QFrame)
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import Qt, QSize

## Structura Codului

Aplicatia este organizata in urmatoarele sectiuni principale:

1. Importul bibliotecilor
2. Variabile globale
3. Clase pentru vizualizare (Spectrograma, Informatii Audio, Spectru de Frecventa)
4. Functii pentru gestionarea fisierelor audio
5. Functii pentru redare si control
6. Efecte audio (filtre si efecte creative)
7. Procesarea si aplicarea efectelor
8. Functii pentru inregistrare
9. Initializare UI
10. Exercitii (TODO-uri)
11. Initializarea aplicatiei

## 1. Importarea Bibliotecilor
În acest bloc, importăm toate bibliotecile necesare pentru a construi playerul audio. 
Folosim:
* PyQt5 pentru interfața grafică
* QMediaPlayer și QMediaPlaylist pentru redarea fișierelor audio
* numpy și sounddevice pentru procesarea audio și aplicarea efectelor
* scipy pentru algoritmi avansați de procesare a semnalelor și filtre
* matplotlib pentru realizarea graficelor
* wave și contextlib pentru manipularea fișierelor WAV
* librosa pentru analiza audio avansată și extragerea datelor din MP3
* mutagen pentru citirea și manipularea metadatelor audio (MP3 și WAV)
* pydub pentru conversii între formate audio și prelucrări de nivel înalt
* traceback pentru debugging și gestionarea excepțiilor


In [3]:
import sys
import os
import numpy as np
import sounddevice as sd
from scipy import signal
import scipy.io.wavfile as wav
import scipy.fftpack as fft
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QHBoxLayout, QWidget,
    QPushButton, QSlider, QFileDialog, QListWidget, QLabel, QComboBox,
    QTabWidget, QDialog, QTableWidget, QTableWidgetItem, QHeaderView, QSizePolicy,
    QFormLayout, QDoubleSpinBox, QSpinBox, QMessageBox
)
from PyQt5.QtCore import Qt, QUrl
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent, QMediaPlaylist
import wave
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import traceback

# Biblioteci pentru MP3 si metadate
import librosa
from mutagen.mp3 import MP3
from mutagen.wave import WAVE

## 2. Variabile globale

Aceste variabile sunt folosite pentru a mentine starea aplicatiei si pentru a facilita accesul din diferite functii.

In [4]:
app = None
window = None
media_player = None
playlist = None
playlist_widget = None
current_audio_file = None
shuffle = False
repeat = False
volume = 1
speed_slider = None
volume_slider = None
progress_slider = None
effects_combo = None
effect_function = None
sample_rate = 44100  # Rata standard de esantionare
audio_data = None
eq_sliders = None  # Lista de slidere pentru equalizer
recording = False
spectrogram_canvas = None  # Canvas pentru spectrograma
frequency_spectrum_canvas = None # Canvas pentru spectrul de frecventa
info_table = None  # Tabel pentru afisarea informatiilor audio

## 3. Clase pentru vizualizare
Aceasta sectiune contine clasele necesare pentru reprezentarea grafica a datelor audio. Una dintre cele mai importante vizualizari este spectrograma, care reprezinta distributia frecventelor in timp.

### 3.1. SpectrogramCanvas
Clasa `SpectrogramCanvas` extinde functionalitatea `FigureCanvas` din Matplotlib pentru a crea o reprezentare vizuala a semnalului audio in domeniul timp-frecventa. Spectrograma permite:

- Vizualizarea componentelor de frecventa ale unui semnal audio
- Identificarea modelelor sonore si a caracteristicilor specifice
- Analiza distributiei energiei in functie de frecventa si timp

Intensitatea culorilor in spectrograma corespunde amplitudinii frecventelor respective - culori mai intense indica o prezenta mai puternica a acelor frecvente.

In [5]:
class SpectrogramCanvas(FigureCanvas):
    """
    Canvas Matplotlib pentru afisarea spectrogramei.
    """
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = self.fig.add_subplot(111)
        
        FigureCanvas.__init__(self, self.fig)
        self.setParent(parent)
        
        FigureCanvas.setSizePolicy(self,
                QSizePolicy.Expanding,
                QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)
        
        self.axes.text(0.5, 0.5, 'Niciun fisier audio incarcat', 
                  horizontalalignment='center', verticalalignment='center',
                  transform=self.axes.transAxes, fontsize=14)
        self.draw()
        
    def plot_spectrogram(self, samples, sample_rate):
        """
        Genereaza si afiseaza spectrograma pentru datele audio.
        
        Parametrii:
        -----------
        samples : numpy.ndarray
            Esantioanele audio
        sample_rate : int
            Rata de esantionare a semnalului
        """
        if samples is None or len(samples) == 0:
            print("Nu exista date pentru spectrograma")
            self.axes.clear()
            self.axes.text(0.5, 0.5, 'Nu exista date audio', 
                      horizontalalignment='center', verticalalignment='center',
                      transform=self.axes.transAxes, fontsize=14)
            self.draw()
            return
            
        print(f"Generare spectrograma pentru {len(samples)} samples la {sample_rate}Hz")
        
        self.axes.clear()
        
        try:
            self.axes.specgram(samples, NFFT=2048, Fs=sample_rate, 
                          noverlap=1024, cmap='viridis', 
                          mode='magnitude', scale='dB')
            self.axes.set_ylabel('Frecventa [Hz]')
            self.axes.set_xlabel('Timp [sec]')
            self.axes.set_title('Spectrograma')
            
            self.fig.tight_layout()
            
            self.draw()
            print("Spectrograma generata cu succes")
        except Exception as e:
            print(f"Eroare la generarea spectrogramei: {e}")
            traceback.print_exc()
            
            self.axes.clear()
            self.axes.text(0.5, 0.5, f'Eroare: {str(e)}', 
                      horizontalalignment='center', verticalalignment='center',
                      transform=self.axes.transAxes, fontsize=10, color='red')
            self.draw()

### 3.2. AudioInfoDialog

Aceasta clasa creeaza un dialog interactiv pentru afisarea detaliilor tehnice ale fisierelor audio. Dialogul prezinta informatii precum:

- Format fisier (WAV/MP3)
- Canale audio (mono/stereo)
- Rata de esantionare
- Durata
- Bitrate
- Metadate (tag-uri ID3 pentru MP3-uri)

Informatiile sunt organizate intr-un tabel cu doua coloane (Proprietate si Valoare) pentru o vizualizare clara si structurata. Utilizatorul poate accesa aceste informatii apasand butonul "Info" din interfata principala.

In [6]:
class AudioInfoDialog(QDialog):
    """
    Dialog pentru afisarea informatiilor detaliate despre fisierul audio.
    """
    def __init__(self, audio_info, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Informatii Fisier Audio")
        self.setGeometry(300, 300, 500, 400)
        
        layout = QVBoxLayout()
        
        self.table = QTableWidget()
        self.table.setColumnCount(2)
        self.table.setHorizontalHeaderLabels(["Proprietate", "Valoare"])
        self.table.horizontalHeader().setSectionResizeMode(1, QHeaderView.Stretch)
        
        self.populate_table(audio_info)
        
        layout.addWidget(self.table)
        
        close_button = QPushButton("Inchide")
        close_button.clicked.connect(self.accept)
        layout.addWidget(close_button)
        
        self.setLayout(layout)
    
    def populate_table(self, info):
        """
        Populeaza tabelul cu informatiile audio.
        
        Parametrii:
        -----------
        info : dict
            Dictionar cu informatii despre fisierul audio
        """
        rows = []
        for key, value in info.items():
            if key != 'metadata':  # Tratam metadatele separat
                rows.append((key, str(value)))
        
        if 'metadata' in info and info['metadata'] and info['metadata'] != "Fara metadate":
            for meta_key, meta_value in info['metadata'].items():
                rows.append((f"Metadata: {meta_key}", str(meta_value)))
                
        self.table.setRowCount(len(rows))
        for i, (key, value) in enumerate(rows):
            self.table.setItem(i, 0, QTableWidgetItem(key))
            self.table.setItem(i, 1, QTableWidgetItem(value))

### 3.3. FrequencySpectrumCanvas
Aceasta clasa implementeaza vizualizarea spectrului de frecventa al unui semnal audio. In contrast cu spectrograma (care arata distributia frecventelor in timp), spectrul de frecventa prezinta amplitudinea fiecarei componente de frecventa la un moment dat.

Spectrul de frecventa este generat utilizand Transformata Fourier Rapida (FFT), care descompune semnalul audio in componentele sale de frecventa. Aceasta permite:

- Identificarea frecventelor dominante dintr-un semnal audio
- Analiza distributiei energiei pe banda de frecventa
- Detectarea componentelor armonice ale sunetelor

Implementarea aplica o fereastra Hamming pentru a reduce efectele de scurgere spectrala si optimizeaza procesarea pentru semnale audio lungi prin esantionare inteligenta.

In [7]:
class FrequencySpectrumCanvas(FigureCanvas):
    """
    Canvas Matplotlib pentru afisarea spectrului de frecventa.
    """
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = self.fig.add_subplot(111)
        
        FigureCanvas.__init__(self, self.fig)
        self.setParent(parent)
        
        FigureCanvas.setSizePolicy(self,
                QSizePolicy.Expanding,
                QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)
        
        self.axes.text(0.5, 0.5, 'Niciun fisier audio incarcat', 
                  horizontalalignment='center', verticalalignment='center',
                  transform=self.axes.transAxes, fontsize=14)
        self.draw()
        
    def plot_frequency_spectrum(self, samples, sample_rate):
        """
        Genereaza si afiseaza spectrul de frecventa pentru datele audio.
        
        Parametrii:
        -----------
        samples : numpy.ndarray
            Esantioanele audio
        sample_rate : int
            Rata de esantionare a semnalului
        """
        if samples is None or len(samples) == 0:
            print("Nu exista date pentru spectrul de frecventa")
            self.axes.clear()
            self.axes.text(0.5, 0.5, 'Nu exista date audio', 
                      horizontalalignment='center', verticalalignment='center',
                      transform=self.axes.transAxes, fontsize=14)
            self.draw()
            return
            
        print(f"Generare spectru de frecventa pentru {len(samples)} samples la {sample_rate}Hz")
        
        self.axes.clear()
        
        try:
            # Calculam FFT
            # Folosim o fereastra Hamming pentru a reduce artefactele
            windowed_samples = samples * np.hamming(len(samples))
            
            # Luam doar o parte din samples pentru FFT mai rapida si mai clara
            # (maxim 100,000 de samples sau toate daca sunt mai putine)
            max_samples = min(len(windowed_samples), 100000)
            segment = windowed_samples[:max_samples]
            
            # Aplicam FFT
            fft_result = np.fft.rfft(segment)
            # Folosim rfft pentru a obtine doar partea pozitiva a spectrului (mai eficient)
            freqs = np.fft.rfftfreq(len(segment), d=1/sample_rate)
            
            # Calculam magnitudinea in dB pentru o vizualizare mai buna
            magnitude = np.abs(fft_result)
            magnitude_db = 20 * np.log10(magnitude + 1e-10)  # Adaugam o valoare mica pentru a evita log(0)
            
            # Afisam spectrul de frecventa
            self.axes.plot(freqs, magnitude_db)
            self.axes.set_xlabel('Frecventa [Hz]')
            self.axes.set_ylabel('Magnitudine [dB]')
            self.axes.set_title('Spectrul de Frecventa')
            
            # Setam limitele axelor pentru o vizualizare mai buna
            self.axes.set_xlim([0, min(20000, sample_rate/2)])  # Limitam la 20kHz sau Nyquist
            
            # Adaugam o grila pentru a face graficul mai usor de citit
            self.axes.grid(True, alpha=0.3)
            
            # Imbunatatim aspectul
            self.fig.tight_layout()
            
            # Actualizam canvas-ul
            self.draw()
            print("Spectru de frecventa generat cu succes")
        except Exception as e:
            print(f"Eroare la generarea spectrului de frecventa: {e}")
            traceback.print_exc()
            
            # Afisam mesaj de eroare pe grafic
            self.axes.clear()
            self.axes.text(0.5, 0.5, f'Eroare: {str(e)}', 
                      horizontalalignment='center', verticalalignment='center',
                      transform=self.axes.transAxes, fontsize=10, color='red')
            self.draw()

## 4. Funcții pentru gestionarea fișierelor audio
Aceasta sectiune contine functiile necesare pentru manipularea, analiza si extragerea datelor din fisierele audio. Aceste functii permit lucrul atat cu fisiere WAV cat si MP3, oferind diverse operatii precum:

- Extragerea informatiilor tehnice (metadata, durata, rata de esantionare)
- Citirea datelor audio pentru prelucrare
- Conversia intre diferite formate
- Analiza caracteristicilor audio

### 4.1. Extragerea și afișarea informațiilor audio
Functia `get_audio_info` analizeaza un fisier audio si extrage toate informatiile tehnice relevante, returnand un dictionar structurat cu aceste date. Informatiile sunt diferentiate in functie de formatul fisierului (WAV sau MP3).

In [8]:
def get_audio_info(file_path):
    """
    Extrage si afiseaza detaliile tehnice ale unui fisier audio.
    Suporta atat fisiere WAV cat si MP3.
    
    Parametrii:
    -----------
    file_path : str
        Calea catre fisierul audio
        
    Returneaza:
    ----------
    dict
        Un dictionar cu informatiile despre fisierul audio
    """
    info = {}
    
    # Verificam extensia fisierului
    ext = os.path.splitext(file_path)[1].lower()
    
    try:
        # Obtinem dimensiunea fisierului
        info['file_size'] = os.path.getsize(file_path)
        info['file_size_mb'] = round(info['file_size'] / (1024 * 1024), 2)
        
        if ext == '.wav':
            try:
                with wave.open(file_path, 'rb') as wav_file:
                    info['format'] = 'WAV'
                    info['channels'] = wav_file.getnchannels()
                    info['sample_width'] = wav_file.getsampwidth() * 8  # convertim la biti
                    info['sample_rate'] = wav_file.getframerate()
                    info['frames'] = wav_file.getnframes()
                    info['duration'] = info['frames'] / info['sample_rate']
                    info['bitrate'] = int(info['sample_rate'] * info['sample_width'] * info['channels'])
                
                # Incarcam metadatele utilizand mutagen
                try:
                    wave_data = WAVE(file_path)
                    if hasattr(wave_data, 'tags'):
                        info['metadata'] = wave_data.tags
                    else:
                        info['metadata'] = "Fara metadate"
                except Exception as e:
                    info['metadata_error'] = str(e)
                    
            except Exception as e:
                info['error'] = f"Eroare la citirea fisierului WAV: {str(e)}"
                
        elif ext == '.mp3':
            try:
                # Folosim mutagen pentru MP3
                mp3_data = MP3(file_path)
                
                info['format'] = 'MP3'
                info['channels'] = mp3_data.info.channels
                info['sample_rate'] = mp3_data.info.sample_rate
                info['duration'] = mp3_data.info.length
                info['bitrate'] = mp3_data.info.bitrate
                
                # Extragem metadatele (ID3 tags)
                if mp3_data.tags:
                    metadata = {}
                    for key, value in mp3_data.tags.items():
                        metadata[key] = str(value)
                    info['metadata'] = metadata
                else:
                    info['metadata'] = "Fara metadate"
                    
            except Exception as e:
                info['error'] = f"Eroare la citirea fisierului MP3: {str(e)}"
        else:
            info['error'] = f"Format de fisier nesuportat: {ext}"
    except Exception as e:
        info['error'] = f"Eroare generala: {str(e)}"
    
    return info

### 4.2. Manipularea fișierelor audio
Functia `extract_audio_data` reprezinta nucleul sistemului de procesare audio, fiind responsabila pentru citirea efectiva a datelor audio din fisiere. Aceasta functie:

- Gestioneaza atat fisiere WAV cat si MP3
- Ofera optiunea de a pastra formatele stereo sau de a converti la mono
- Normalizeaza datele pentru procesare ulterioara
- Include gestionarea erorilor robusta pentru a preveni intreruperile aplicatiei

Functia utilizeaza librarii specializate pentru fiecare format (scipy.io.wavfile pentru WAV si librosa pentru MP3) si returneaza datele intr-un format standardizat pentru a fi utilizate cu usurinta in toate celelalte componente ale aplicatiei.

In [9]:
def extract_audio_data(file_path, keep_stereo=False):
    """
    Extrage datele audio dintr-un fisier pentru vizualizare si procesare.
    Suporta atat WAV cat si MP3.
    
    Parametrii:
    -----------
    file_path : str
        Calea catre fisierul audio
    keep_stereo : bool
        Daca este True, pastreaza canalele stereo (daca exista)
        
    Returneaza:
    --------
    tuple
        Un tuplu (samples, sample_rate), unde samples este un numpy.ndarray si
        sample_rate este rata de esantionare in Hz
    """
    try:
        # Verificam daca fisierul exista
        if not os.path.exists(file_path):
            print(f"Fisierul nu exista: {file_path}")
            return None, None
            
        ext = os.path.splitext(file_path)[1].lower()
        
        if ext == '.wav':
            try:
                # Citim direct fisierul WAV
                sample_rate, samples = wav.read(file_path)
                
                # Debug info - afiseaza informatii despre forma datelor
                print(f"WAV: Forma initiala a datelor: {samples.shape}, tip: {samples.dtype}")
                
                # Daca avem stereo si nu vrem sa pastram canalele stereo, convertim la mono pentru vizualizare
                if len(samples.shape) > 1 and not keep_stereo:
                    print(f"Convertesc audio stereo la mono (canale: {samples.shape[1]})")
                    samples = np.mean(samples, axis=1).astype(np.int16)
                    print(f"WAV: Forma dupa conversie la mono: {samples.shape}")
                
                return samples, sample_rate
            except Exception as e:
                print(f"Eroare la citirea fisierului WAV: {e}")
                traceback.print_exc()
                return None, None
                
        elif ext == '.mp3':
            try:
                # Folosim librosa pentru a extrage datele audio din MP3
                if keep_stereo:
                    # Pentru MP3, daca vrem stereo
                    samples, sample_rate = librosa.load(file_path, sr=None, mono=False)
                    print(f"MP3: Forma initiala a datelor stereo: {samples.shape}, tip: {samples.dtype}")
                    
                    # librosa returneaza un array de forma (canale, samples) dar noi avem nevoie de (samples, canale)
                    if len(samples.shape) > 1:
                        samples = samples.T
                        print(f"MP3: Forma dupa transpozitie: {samples.shape}")
                else:
                    # Pentru vizualizare, de obicei folosim mono
                    samples, sample_rate = librosa.load(file_path, sr=None, mono=True)
                    print(f"MP3: Forma initiala a datelor mono: {samples.shape}, tip: {samples.dtype}")
                
                # Convertim la int16 pentru compatibilitate
                samples = (samples * 32767).astype(np.int16)
                print(f"MP3: Forma finala dupa conversie la int16: {samples.shape}")
                
                return samples, sample_rate
            except Exception as e:
                print(f"Eroare la citirea fisierului MP3: {str(e)}")
                traceback.print_exc()
                return None, None
                
        else:
            print(f"Format nesuportat: {ext}")
            return None, None
            
    except Exception as e:
        print(f"Eroare generala la procesarea fisierului: {str(e)}")
        traceback.print_exc()
        return None, None

### 4.3. Interfață pentru fișiere
Aceasta sectiune contine functii pentru gestionarea fisierelor audio in interfata grafica a aplicatiei. Functiile principale sunt:

- `open_audio_file()`: Deschide un dialog pentru selectarea si incarcarea fisierelor audio
- `remove_selected_song()`: Elimina fisiere din playlist si, optional, de pe disc
- `play_selected_song()`: Gestioneaza redarea fisierelor selectate
- `show_audio_info()`: Afiseaza informatii tehnice despre fisierul curent
- `compare_stereo_channels()`: Analizeaza si compara canalele stereo
- `update_visualizations()`: Actualizeaza reprezentarile vizuale ale semnalului audio

Aceste functii lucreaza impreuna pentru a oferi o experienta completa de gestionare a fisierelor audio, permitand utilizatorului sa manipuleze colectia de fisiere si sa obtina informatii detaliate despre acestea.

In [10]:
### 6.3. Interfata pentru fisiere
def open_audio_file():
    """
    Deschide un dialog pentru selectarea unui fisier audio si il adauga in playlist.
    Actualizeaza atat playlist-ul intern cat si interfata vizuala.
    Actualizeaza variabila current_audio_file pentru a permite aplicarea efectelor.
    """
    global current_audio_file
    file_path, _ = QFileDialog.getOpenFileName(window, "Selecteaza Fisier Audio", "", "Audio Files (*.mp3 *.wav)")
    if file_path:
        current_audio_file = file_path
        
        # Verificam formatul fisierului
        ext = os.path.splitext(file_path)[1].lower()
        if ext == '.mp3':
            print(f"Fisier MP3 incarcat: {file_path}")
        else:
            print(f"Fisier WAV incarcat: {file_path}")
            
        # Verificam daca fisierul exista inainte de a-l adauga
        if not os.path.exists(file_path):
            print(f"EROARE: Fisierul {file_path} nu exista!")
            return
            
        media_content = QMediaContent(QUrl.fromLocalFile(file_path))
        playlist.addMedia(media_content)
        playlist_widget.addItem(os.path.basename(file_path))
        
        # Verificam daca media_content este valid
        print(f"Media content valid: {media_content.isNull() == False}")
        
        # Actualizam vizualizarile cand se incarca un fisier nou
        update_visualizations()

def remove_selected_song():
    """
    Elimina melodia selectata din playlist si din interfata vizuala.
    Daca este un fisier de inregistrare, il sterge si de pe disc.
    """
    selected_item = playlist_widget.currentItem()
    if selected_item:
        file_name = selected_item.text()
        file_path = os.path.join(os.getcwd(), file_name)

        # Elimina fisierul din playlist si interfata
        row = playlist_widget.row(selected_item)
        playlist.removeMedia(row)
        playlist_widget.takeItem(row)

        # Sterge fisierul de pe disc doar daca este un fisier de inregistrare
        if file_name.startswith("recorded_audio_") and os.path.exists(file_path):
            try:
                os.remove(file_path)
            except Exception as e:
                print(f"Eroare la stergerea fisierului: {e}")

def play_selected_song():
    """
    Reda melodia selectata din playlist.
    Se activeaza atunci cand utilizatorul face dublu-click pe un element din lista.
    Actualizeaza variabila current_audio_file pentru a permite aplicarea efectelor.
    """
    global current_audio_file
    
    selected = playlist_widget.currentRow()
    if selected != -1:
        # Obtinem numele fisierului selectat din playlist widget
        selected_file_name = playlist_widget.item(selected).text()
        
        # Verificam daca e un fisier inregistrat sau incarcat
        if selected_file_name.startswith("recorded_audio_"):
            # Pentru fisier inregistrat, folosim calea relativa din directorul curent
            current_audio_file = os.path.abspath(selected_file_name)
        else:
            # Pentru celelalte fisiere, trebuie sa obtinem URL-ul media
            url = playlist.media(selected).canonicalUrl()
            if url.isValid():
                current_audio_file = url.toLocalFile()
        
        # Verificam daca fisierul exista
        if not os.path.exists(current_audio_file):
            print(f"EROARE: Fisierul {current_audio_file} nu exista!")
            return
            
        print(f"Redare fisier: {current_audio_file}")
        
        # Verificam formatul fisierului
        ext = os.path.splitext(current_audio_file)[1].lower()
        if ext == '.mp3':
            print("Redare MP3...")
        else:
            print("Redare WAV...")
            
        # Setam indexul si pornim redarea
        playlist.setCurrentIndex(selected)
        media_player.play()
        
        # Afisam starea player-ului
        print(f"Stare player dupa play: {media_player.state()}")
        if media_player.state() != QMediaPlayer.PlayingState:
            # Daca player-ul nu este in stare de redare, afisam eroarea
            error = media_player.error()
            print(f"Eroare media player: {error}")
            
        # Actualizam vizualizarile pentru fisierul curent
        update_visualizations()

def show_audio_info():
    """
    Afiseaza un dialog cu informatii detaliate despre fisierul audio curent.
    """
    global current_audio_file
    
    if current_audio_file and os.path.exists(current_audio_file):
        # Obtinem informatiile audio
        audio_info = get_audio_info(current_audio_file)
        
        # Afisam dialogul cu informatii
        dialog = AudioInfoDialog(audio_info, window)
        dialog.exec_()
    else:
        print("Nu exista un fisier audio selectat sau fisierul nu exista")

def compare_stereo_channels():
    """
    Compara canalele stereo ale fisierului audio curent.
    Afiseaza informatii despre diferentele dintre canale.
    """
    global current_audio_file
    
    if current_audio_file:
        print(f"Analizez fisierul pentru comparare canale: {current_audio_file}")
        # Folosim keep_stereo=True pentru a pastra canalele stereo
        samples, sample_rate = extract_audio_data(current_audio_file, keep_stereo=True)
        
        if samples is None:
            print("Nu s-au putut extrage date audio")
            return
            
        print(f"Forma datelor extrase: {samples.shape}, tip: {samples.dtype}")
        
        # Verificam forma array-ului pentru a determina daca este stereo
        if len(samples.shape) > 1 and samples.shape[1] > 1:
            print(f"Fisierul are {samples.shape[1]} canale")
            channel_left = samples[:, 0]  # Canalul stang
            channel_right = samples[:, 1]  # Canalul dreapta
            
            # Calculam diferentele
            difference = np.abs(channel_left - channel_right)
            
            print("=== Comparare Canale Stereo ===")
            print(f"Diferenta maxima intre canale: {np.max(difference)}")
            print(f"Diferenta medie intre canale: {np.mean(difference)}")
            print(f"Diferenta minima intre canale: {np.min(difference)}")
            
            # Adaugam informatii suplimentare utile
            print(f"Corelatia dintre canale: {np.corrcoef(channel_left, channel_right)[0,1]}")
            
            # Afisam un mesaj personalizat bazat pe diferenta medie
            mean_diff = np.mean(difference)
            if mean_diff < 10:
                print("Canalele sunt aproape identice. Probabil este un fisier mono convertit la stereo.")
            elif mean_diff < 1000:
                print("Canalele au diferente moderate. Este un fisier stereo tipic.")
            else:
                print("Canalele au diferente semnificative. Este un fisier stereo cu canale distincte.")
        else:
            print("Fisierul audio nu este stereo (are un singur canal).")
            print(f"Detalii despre fisier: {current_audio_file}")
            # Determinam formatul fisierului
            ext = os.path.splitext(current_audio_file)[1].lower()
            print(f"Format fisier: {ext}")
            
            # Incercam sa extragem mai multe informatii despre fisier
            try:
                if ext == '.wav':
                    with wave.open(current_audio_file, 'rb') as wav_file:
                        channels = wav_file.getnchannels()
                        print(f"Numar de canale conform wave: {channels}")
                elif ext == '.mp3':
                    mp3_data = MP3(current_audio_file)
                    channels = mp3_data.info.channels
                    print(f"Numar de canale conform MP3: {channels}")
            except Exception as e:
                print(f"Eroare la obtinerea informatiilor detaliate: {e}")

def update_visualizations():
    """
    Actualizeaza spectrograma si spectrul de frecventa pentru fisierul audio curent.
    Adaugata gestionarea erorilor si debugging pentru MP3.
    """
    global current_audio_file, spectrogram_canvas, frequency_spectrum_canvas
    
    if current_audio_file and os.path.exists(current_audio_file):
        print(f"Actualizare vizualizari pentru: {current_audio_file}")
        
        if spectrogram_canvas is None:
            print("EROARE: spectrogram_canvas nu este initializat!")
            return
            
        if frequency_spectrum_canvas is None:
            print("EROARE: frequency_spectrum_canvas nu este initializat!")
            return
            
        try:
            # Extragem datele audio pentru vizualizare (folosim mono pentru vizualizari)
            samples, sample_rate = extract_audio_data(current_audio_file, keep_stereo=False)
            
            if samples is not None and sample_rate is not None:
                print(f"Generare vizualizari: {len(samples)} samples la {sample_rate}Hz")
                
                # Ne asiguram ca datele sunt in formatul corect pentru vizualizare
                if len(samples.shape) > 1:
                    print(f"AVERTISMENT: Date cu multiple canale ({samples.shape}). Convertesc la mono pentru vizualizare.")
                    samples = np.mean(samples, axis=1).astype(np.int16)
                
                # Adaugam verificari suplimentare pentru datele audio
                if len(samples) == 0:
                    print("EROARE: Nu exista samples pentru vizualizare!")
                    return
                    
                if np.all(samples == 0):
                    print("AVERTISMENT: Toate valorile sunt zero!")
                
                # Generam spectrograma
                try:
                    print("Generare spectrograma...")
                    spectrogram_canvas.plot_spectrogram(samples, sample_rate)
                    print("Spectrograma generata cu succes")
                except Exception as e:
                    print(f"Eroare la generarea spectrogramei: {e}")
                    traceback.print_exc()
                
                # Generam spectrul de frecventa
                try:
                    print("Generare spectru de frecventa...")
                    frequency_spectrum_canvas.plot_frequency_spectrum(samples, sample_rate)
                    print("Spectru de frecventa generat cu succes")
                except Exception as e:
                    print(f"Eroare la generarea spectrului de frecventa: {e}")
                    traceback.print_exc()
            else:
                print("Nu s-au putut obtine date audio pentru vizualizari")
        except Exception as e:
            print(f"Eroare la actualizarea vizualizarilor: {e}")
            traceback.print_exc()
    else:
        if not current_audio_file:
            print("Nu exista un fisier audio curent pentru vizualizare")
        elif not os.path.exists(current_audio_file):
            print(f"Fisierul audio curent nu exista: {current_audio_file}")
        else:
            print("Alta eroare necunoscuta la actualizarea vizualizarilor")

## 5. Funcții pentru redare și control
Aceasta sectiune contine functiile necesare pentru controlul redarii audio si manipularea playlist-ului. Aceste functii permit utilizatorului sa interactioneze cu fisierele audio prin interfata grafica, oferind control asupra redarii, volumului, vitezei si modurilor de organizare a playlist-ului.

### 5.1. Funcții de control al redării
Functiile din aceasta categorie gestioneaza starea de redare a player-ului audio, oferind operatii fundamentale precum:

- Comutarea intre redare si pauza
- Oprirea completa a redarii
- Activarea/dezactivarea redarii aleatorii
- Activarea/dezactivarea repetarii
- Ajustarea volumului si a vitezei de redare

Aceste controale sunt conectate direct la interfata grafica si permit utilizatorului sa manipuleze experienta de ascultare in timp real.

In [11]:
def toggle_play():
    """
    Comuta intre starile de redare si pauza.
    Daca playerul ruleaza, se pune pauza; daca este in pauza, se reia redarea.
    """
    if media_player.state() == QMediaPlayer.PlayingState:
        media_player.pause()
    else:
        media_player.play()

def stop_audio():
    """Opreste complet redarea audio si reseteaza pozitia la inceput."""
    media_player.stop()

def toggle_shuffle():
    """
    Comuta intre modurile de redare secvential si aleatoriu.
    Actualizeaza variabila globala si setarea playlist-ului.
    """
    global shuffle
    shuffle = not shuffle
    mode = QMediaPlaylist.Random if shuffle else QMediaPlaylist.Sequential
    playlist.setPlaybackMode(mode)

def toggle_repeat():
    """
    Comuta intre modurile de redare unica si repetitiva.
    Actualizeaza variabila globala si setarea playlist-ului.
    """
    global repeat
    repeat = not repeat
    mode = QMediaPlaylist.Loop if repeat else QMediaPlaylist.Sequential
    playlist.setPlaybackMode(mode)

def change_volume(value):
    """
    Modifica volumul de redare in functie de valoarea primita de la slider.
    Converteste valoarea din interval 0-100 la interval 0.0-1.0.
    
    Parameters:
    -----------
    value : int
        Valoarea volumului (0-100)
    """
    global volume
    volume = value / 100.0
    media_player.setVolume(value)

def change_speed(value):
    """
    Modifica viteza de redare in functie de valoarea primita de la slider.
    Valoarea 100 reprezinta viteza normala.
    
    Parameters:
    -----------
    value : int
        Valoarea vitezei (50-200, unde 100 este 1x)
    """
    speed = value / 100
    media_player.setPlaybackRate(speed)

### 5.2. Gestionarea progresului redării
Aceste functii implementeaza controlul pozitiei de redare in fisierul audio, permitand:

- Actualizarea automata a slider-ului de progres pe masura ce fisierul este redat
- Navigarea manuala in interiorul fisierului audio prin deplasarea slider-ului

Functiile lucreaza impreuna pentru a sincroniza starea interna a player-ului media cu interfata grafica, asigurand o experienta consistenta pentru utilizator. Slider-ul de progres afiseaza pozitia curenta ca procent din durata totala a fisierului, iar utilizatorul poate sari la orice moment din fisier prin interactiunea cu acest control.

In [12]:
def update_progress(position):
    """
    Actualizeaza pozitia slider-ului de progres in functie de pozitia curenta de redare.
    
    Paramaterii:
    -----------
    position : int
        Pozitia curenta in milisecunde
    """
    duration = media_player.duration()
    if duration > 0:
        progress_slider.setValue(int((position / duration) * 100))

def set_position(position):
    """
    Seteaza pozitia de redare cand utilizatorul muta manual slider-ul de progres.
    Converteste valoarea procentuala in milisecunde.
    
    Paramaterii:
    -----------
    position : int
        Pozitia dorita (0-100)
    """
    duration = media_player.duration()
    media_player.setPosition(int((position / 100) * duration))

## 6. Efecte audio
Aceasta sectiune implementeaza diverse efecte si filtre pentru procesarea audio, permitand transformarea si manipularea semnalelor audio digitale. Biblioteca include filtre de baza, filtre FIR (Finite Impulse Response), precum si efecte creative avansate.

### 6.1. Baza filtrelor
Aceasta subsectiune contine implementari fundamentale pentru procesarea semnalelor, cum ar fi media mobila (running mean), care poate fi utilizata pentru netezirea semnalului audio si reducerea zgomotului.

In [13]:
# http://stackoverflow.com/questions/13728392/moving-average-or-running-mean
def running_mean(x, windowSize):
    """
    Calculeaza media mobila pentru un array.
    Aceasta functie realizeaza o netezire a semnalului audio.
    
    Paramaterii:
    -----------
    x : numpy.ndarray
        Array-ul pentru care se calculeaza media mobila
    windowSize : int
        Dimensiunea ferestrei pentru calculul mediei
        
    Returneaza:
    --------
    numpy.ndarray
        Array-ul cu valorile mediate
    """
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[windowSize:] - cumsum[:-windowSize]) / windowSize

### 6.2. Filtre FIR
Filtrele FIR (Finite Impulse Response) sunt componente esentiale in procesarea digitala a semnalelor audio. Aceste filtre au raspuns finit la impuls si sunt implementate prin convolutia semnalului cu un set de coeficienti. Avantajele filtrelor FIR includ:

- Stabilitate garantata (nu au feedback)
- Faza liniara (cand sunt proiectate simetric)
- Implementare simpla si eficienta

In [14]:
# https://fiiir.com
def fir_low_pass(audio_data, sample_rate, cutoff_freq, N, outputType=np.int16):
    """
    Aplica un filtru trece-jos FIR la datele audio.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    cutoff_freq : float
        Frecventa de taiere in Hz
    N : int
        Ordinul filtrului (numarul de coeficienti)
    outputType : numpy.dtype, optional
        Tipul de date pentru iesire (default: np.int16)
        
    Returneaza:
    --------
    numpy.ndarray
        Datele audio filtrate
    """
    # Normalizam frecventa de taiere
    cutoff_freq = cutoff_freq / sample_rate

    # Calculam filtrul sinc
    h = np.sinc(2 * cutoff_freq * (np.arange(N) - (N - 1) / 2.))
    # Aplicam fereastra Hamming
    h *= np.hamming(N)
    # Normalizam pentru a obtine un castig unitar
    h /= np.sum(h)
    
    # Aplicam filtrul prin convolutie
    audio_filtered = np.convolve(audio_data, h)
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(audio_filtered))
    if max_val > 32767:
        audio_filtered = audio_filtered * (32767 / max_val)
    
    return np.clip(audio_filtered, -32768, 32767).astype(outputType)

# https://fiiir.com
def fir_band_pass(audio_data, sample_rate, low_cutoff, high_cutoff, N_low, N_high, outputType=np.int16):
    """
    Aplica un filtru trece-banda FIR la datele audio.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    low_cutoff : float
        Frecventa inferioara de taiere in Hz
    high_cutoff : float
        Frecventa superioara de taiere in Hz
    N_low : int
        Ordinul filtrului trece-sus
    N_high : int
        Ordinul filtrului trece-jos
    outputType : numpy.dtype, optional
        Tipul de date pentru iesire (default: np.int16)
        
    Returneaza:
    --------
    numpy.ndarray
        Datele audio filtrate
    """
    # Normalizam frecventele de taiere
    low_cutoff = low_cutoff / sample_rate
    high_cutoff = high_cutoff / sample_rate

    # Calculam filtrul trece-jos cu frecventa de taiere high_cutoff
    hlpf = np.sinc(2 * high_cutoff * (np.arange(N_high) - (N_high - 1) / 2.))
    hlpf *= np.blackman(N_high)
    hlpf /= np.sum(hlpf)
    
    # Calculam filtrul trece-sus cu frecventa de taiere low_cutoff
    hhpf = np.sinc(2 * low_cutoff * (np.arange(N_low) - (N_low - 1) / 2.))
    hhpf *= np.blackman(N_low)
    hhpf /= np.sum(hhpf)
    hhpf = -hhpf
    hhpf[int((N_low - 1) / 2)] += 1
    
    # Convolutie intre filtrele trece-jos si trece-sus pentru a obtine filtrul trece-banda
    h = np.convolve(hlpf, hhpf)
    
    # Aplicam filtrul prin convolutie
    audio_filtered = np.convolve(audio_data, h)
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(audio_filtered))
    if max_val > 32767:
        audio_filtered = audio_filtered * (32767 / max_val)
    
    return np.clip(audio_filtered, -32768, 32767).astype(outputType)

# https://fiiir.com
def fir_band_reject(audio_data, sample_rate, low_cutoff, high_cutoff, N_low, N_high, outputType=np.int16):
    """
    Aplica un filtru opreste-banda FIR la datele audio.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    low_cutoff : float
        Frecventa inferioara de taiere in Hz
    high_cutoff : float
        Frecventa superioara de taiere in Hz
    N_low : int
        Ordinul filtrului trece-jos
    N_high : int
        Ordinul filtrului trece-sus
    outputType : numpy.dtype, optional
        Tipul de date pentru iesire (default: np.int16)
        
    Returneaza:
    --------
    numpy.ndarray
        Datele audio filtrate
    """
    # Normalizam frecventele de taiere
    low_cutoff = low_cutoff / sample_rate
    high_cutoff = high_cutoff / sample_rate

    # Calculam filtrul trece-jos cu frecventa de taiere low_cutoff
    hlpf = np.sinc(2 * low_cutoff * (np.arange(N_low) - (N_low - 1) / 2.))
    hlpf *= np.blackman(N_low)
    hlpf /= np.sum(hlpf)
    
    # Calculam filtrul trece-sus cu frecventa de taiere high_cutoff
    hhpf = np.sinc(2 * high_cutoff * (np.arange(N_high) - (N_high - 1) / 2.))
    hhpf *= np.blackman(N_high)
    hhpf /= np.sum(hhpf)
    hhpf = -hhpf
    hhpf[int((N_high - 1) / 2)] += 1
    
    # Adaugam ambele filtre
    if N_high >= N_low:
        h = hhpf
        h[int((N_high - N_low) / 2) : int((N_high - N_low) / 2 + N_low)] += hlpf
    else:
        h = hlpf
        h[int((N_low - N_high) / 2) : int((N_low - N_high) / 2 + N_high)] += hhpf
    
    # Aplicam filtrul prin convolutie
    audio_filtered = np.convolve(audio_data, h)
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(audio_filtered))
    if max_val > 32767:
        audio_filtered = audio_filtered * (32767 / max_val)
    
    return np.clip(audio_filtered, -32768, 32767).astype(outputType)

### 6.3. Efecte simple
Aceasta sectiune implementeaza efecte audio simple pentru prelucrarea creativa a semnalelor. Biblioteca include urmatoarele efecte:

- **Bass Boost** - Amplifica frecventele joase (bass) folosind un filtru specializat
- **Reverb** - Simuleaza reverberatia naturala a sunetului in spatii inchise
- **Echo** - Creeaza efecte de ecou cu multiple repetitii si atenuari customizabile
- **Speed Up** - Modifica viteza de redare cu optiunea de a pastra inaltimea originala

Aceste efecte pot fi aplicate individual sau combinate pentru a crea transformari complexe ale semnalelor audio, oferind utilizatorilor posibilitati extinse de editare si procesare creativa.

In [15]:
# https://stackoverflow.com/questions/1794010/how-to-use-numpy-with-portaudio-to-extract-bass-mid-treble
def apply_bass_boost(audio_data, sample_rate, boost_factor=2.5, cutoff_freq=150.0):
    """
    Un bass boost imbunatatit care foloseste un filtru trece-jos pentru a accentua doar frecventele joase
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    boost_factor : float
        Factorul de amplificare pentru bass (intre 1.0 si 5.0 recomandat)
    cutoff_freq : float
        Frecventa de taiere pentru bass (Hz)
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu bass-ul amplificat
    """

    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    
    # Proiectam filtrul butter trece-jos
    nyq = 0.5 * sample_rate
    cutoff_norm = cutoff_freq / nyq
    b, a = signal.butter(4, cutoff_norm, btype='low')
    
    # Aplicam filtrul pentru a obtine doar frecventele joase
    bass_frequencies = signal.lfilter(b, a, audio_float)
    
    # Amplificam frecventele joase si le adaugam inapoi la semnalul original
    boosted_audio = audio_float + (boost_factor - 1) * bass_frequencies
    
    # Normalizam pentru a preveni clipping
    # Clipping apare cand amplitudinea semnalului depaseste limitele permise (-32768 pana la 32767 pentru int16)
    max_val = np.max(np.abs(boosted_audio))
    if max_val > 32767:
        boosted_audio = boosted_audio * (32767 / max_val)
    
    return np.clip(boosted_audio, -32768, 32767).astype(np.int16)

# https://www.youtube.com/watch?v=eFSK09499IY
def apply_reverb(audio_data, sample_rate, room_size=0.75, damping=0.5, wet_level=0.3):
    """
    Aplica un efect de reverberatie (ecou spatial) la datele audio.
    Simuleaza sunetul intr-o incapere cu multiple reflectii.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    room_size : float
        Marimea virtuala a camerei (0.0-1.0)
    damping : float
        Atenuarea reflectiilor (0.0-1.0)
    wet_level : float
        Nivelul efectului (0.0-1.0)
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu efect de reverberatie
    """

    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    
    # Definim mai multe reflectii cu timpi si amplitudini diferite
    num_reflections = 8  # Numarul de reflectii simulate
    reverb_signal = np.zeros_like(audio_float)
    
    # Adaugam semnalul original
    reverb_signal += audio_float * (1.0 - wet_level)
    
    # Generam multiple reflectii cu delay-uri si atenuari diferite
    for i in range(num_reflections):
        # Calculam delay-ul (mai lung pentru room_size mai mare)
        delay_ms = 50 + (room_size * 150 * (i + 1))
        delay_samples = int(sample_rate * delay_ms / 1000)
        
        # Calculam factorul de decay (mai mare pentru damping mai mic)
        decay = wet_level * (1.0 - damping) ** (i+1) / (i+1)
        
        # Aplicam delay-ul si adaugam la semnalul de reverb
        if delay_samples < len(audio_float):
            # Cream reflectia
            reflection = np.zeros_like(audio_float)
            reflection[delay_samples:] = audio_float[:-delay_samples] * decay
            reverb_signal += reflection
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(reverb_signal))
    if max_val > 32767:
        reverb_signal = reverb_signal * (32767 / max_val)
    
    return np.clip(reverb_signal, -32768, 32767).astype(np.int16)

# https://www.thecodingforums.com/threads/how-to-add-echo-to-wav-file-in-python.974511/
def apply_echo(audio_data, sample_rate, delays_ms=[200, 400], decays=[0.5, 0.25]):
    """
    Un efect de echo imbunatatit cu multiple echo-uri
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    delays_ms : list
        Lista de timpi de delay in milisecunde pentru fiecare echo
    decays : list
        Lista de factori de decay pentru fiecare echo
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu efect de echo
    """

    # Verificam ca avem acelasi numar de delays si decays
    if len(delays_ms) != len(decays):
        raise ValueError("Listele delays_ms si decays trebuie sa aiba aceeasi lungime")
    
    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    echo_signal = np.copy(audio_float)
    
    # Aplicam fiecare echo
    for delay_ms, decay in zip(delays_ms, decays):
        delay_samples = int(sample_rate * delay_ms / 1000)
        
        # Verificam ca delay-ul nu e prea mare
        if delay_samples >= len(audio_float):
            continue
        
        # Cream echo-ul si il adaugam la semnalul final
        echo = np.zeros_like(audio_float)
        echo[delay_samples:] = audio_float[:-delay_samples] * decay
        echo_signal += echo
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(echo_signal))
    if max_val > 32767:
        echo_signal = echo_signal * (32767 / max_val)
    
    return np.clip(echo_signal, -32768, 32767).astype(np.int16)

# https://github.com/gaganbahga/time_stretch
def apply_speed_up(audio_data, sample_rate, speed_factor=1.2, preserve_pitch=True):
    """
    O functie pentru modificarea vitezei audio care pastreaza calitatea
    si ofera optiunea de a mentine pitch-ul original.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    speed_factor : float
        Factorul de accelerare (1.0 = viteza normala, 2.0 = viteza dubla)
    preserve_pitch : bool
        Daca se pastreaza pitch-ul original (True) sau se permite modificarea lui (False)
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio procesate cu viteza modificata
    """
    
    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    
    if preserve_pitch:
        # Metoda complexa: Transformarea Fourier pe termen scurt (STFT)
        # Aceasta metoda permite modificarea vitezei fara a afecta inaltimea sunetului
        
        # Calculam noua lungime a semnalului
        new_length = int(len(audio_float) / speed_factor)
        
        # Parametri pentru STFT
        hop_length = 256  # Dimensiunea pasului intre ferestre consecutive
        n_fft = 2048      # Dimensiunea ferestrei FFT
        
        # Folosim libraria signal pentru STFT
        f, t, X = signal.stft(audio_float, fs=sample_rate, nperseg=n_fft, 
                            noverlap=n_fft-hop_length, boundary=None)
        
        # Modificam axa temporala prin interpolare
        time_steps = np.arange(X.shape[1])
        new_time_steps = np.linspace(0, X.shape[1] - 1, int(X.shape[1] / speed_factor))
        
        # Redimensionam STFT folosind interpolare
        new_X = np.zeros((X.shape[0], len(new_time_steps)), dtype=X.dtype)
        
        # Interpolam separat partea reala si imaginara pentru a mentine proprietatile complexe
        for i in range(X.shape[0]):
            real_part = np.interp(new_time_steps, time_steps, np.real(X[i, :]))
            imag_part = np.interp(new_time_steps, time_steps, np.imag(X[i, :]))
            new_X[i, :] = real_part + 1j * imag_part
        
        # Aplicam ISTFT pentru a reconstrui semnalul
        _, modified_audio = signal.istft(new_X, fs=sample_rate, nperseg=n_fft,
                                       noverlap=n_fft-hop_length, boundary=None)
        
        # Asiguram-ne ca lungimea este corecta
        if len(modified_audio) > new_length:
            modified_audio = modified_audio[:new_length]
        
    else:
        # Metoda simpla: Resample pentru modificarea vitezei cu schimbarea pitch-ului
        # Acest lucru va creste pitch-ul atunci cand crestem viteza
        new_length = int(len(audio_float) / speed_factor)
        modified_audio = signal.resample(audio_float, new_length)
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(modified_audio))
    if max_val > 32767:
        modified_audio = modified_audio * (32767 / max_val)
    
    # Convertim inapoi la int16
    return np.clip(modified_audio, -32768, 32767).astype(np.int16)

### 6.4. Efecte avansate
Aceasta sectiune implementeaza efecte audio mai complexe care utilizeaza tehnici avansate de procesare a semnalelor digitale pentru a obtine transformari sofisticate ale sunetului. Printre aceste efecte se numara:

- **Noise Cancelling FFT** - Reduce zgomotul de fond prin analiza spectrala si filtrarea componentelor de zgomot
- **Equalizer** - Permite ajustarea fina a spectrului de frecventa prin amplificarea/atenuarea benzilor specifice
- **Tremolo** - Creeaza un efect de modulatie a amplitudinii pentru a obtine o vibratie controlata a sunetului

Aceste efecte avansate ofera utilizatorilor posibilitatea de a aplica transformari profesionale asupra semnalelor audio, permitand nu doar imbunatatirea calitatii dar si crearea de efecte sonore distincte.

In [16]:
# https://earthinversion.com/techniques/signal-denoising-using-fast-fourier-transform/
def apply_noise_cancelling_fft(audio_data, sample_rate, noise_threshold=0.06):
    """
    Aplica un filtru de reducere a zgomotului folosind FFT (Fast Fourier Transform).
    Aceasta functie elimina componentele de frecventa cu amplitudini mici, considerate zgomot.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare
    noise_threshold : float
        Pragul relativ pentru eliminarea zgomotului (0.0-1.0)
        
    Returneaza:
    --------
    numpy.ndarray
        Datele audio filtrate
    """
    # Aplicam transformata Fourier
    transformed = fft.fft(audio_data)
    
    # Calculam magnitudinea fiecarei componente de frecventa
    magnitude = np.abs(transformed)
    
    # Aplicam un prag pentru a elimina frecventele cu amplitudini mici (zgomotul)
    threshold = noise_threshold * np.max(magnitude)
    transformed[magnitude < threshold] = 0

    # Convertim inapoi la domeniul timpului
    filtered_signal = np.real(fft.ifft(transformed)).astype(np.int16)
    
    return filtered_signal


# https://makethatlouder.com/eq-cheat-sheet-settings-and-frequencies-explained/
def apply_equalizer(audio_data, sample_rate, gains=None):
    """
    Aplica un equalizer cu mai multe benzi de frecventa la datele audio.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    gains : list
        Lista de valori de amplificare pentru fiecare banda de frecventa.
        Valori >1 amplifica, valori <1 atenueaza. Valori intre 0.1 si 3.0 sunt recomandate.
        Daca None, se folosesc valorile implicite [1.0, 1.0, 1.0, 1.0, 1.0]
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu equalizer aplicat
    """
    # Definim frecventele centrale pentru un equalizer cu 5 benzi
    # Acoperim spectrul audio de la frecvente joase la inalte
    if gains is None:
        gains = [1.0, 1.0, 1.0, 1.0, 1.0]  # Valori implicite (neutru)
    
    # Asiguram-ne ca avem 5 valori pentru gain
    if len(gains) != 5:
        raise ValueError("Trebuie sa specificati exact 5 valori pentru gain-uri")
    
    # Definim benzile de frecventa (Hz) - puteti ajusta aceste valori
    bands = [
        (20, 250),      # Bass - frecvente joase
        (250, 500),     # Low-mids
        (500, 2000),    # Mids
        (2000, 5000),   # Upper-mids
        (5000, 20000)   # Highs - frecvente inalte
    ]
    
    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    
    # Aplicam equalizer-ul folosind filtre trece-banda
    equalized_audio = np.zeros_like(audio_float, dtype=np.float32)
    
    # Pentru fiecare banda aplicam un filtru si amplificam/atenuam
    for i, ((low_freq, high_freq), gain) in enumerate(zip(bands, gains)):
        # Normalizam frecventele
        nyq = 0.5 * sample_rate
        low_norm = low_freq / nyq
        high_norm = high_freq / nyq
        
        # Pentru banda de bass folosim un filtru trece-jos
        if i == 0:
            b, a = signal.butter(2, high_norm, btype='lowpass')
        # Pentru banda de inalte folosim un filtru trece-sus
        elif i == len(bands) - 1:
            b, a = signal.butter(2, low_norm, btype='highpass')
        # Pentru benzile de mijloc folosim filtre trece-banda
        else:
            b, a = signal.butter(2, [low_norm, high_norm], btype='bandpass')
        
        # Aplicam filtrul
        filtered_band = signal.lfilter(b, a, audio_float)
        
        # Aplicam gain-ul si adaugam la rezultatul final
        equalized_audio += filtered_band * gain
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(equalized_audio))
    if max_val > 32767:
        equalized_audio = equalized_audio * (32767 / max_val)
    
    return np.clip(equalized_audio, -32768, 32767).astype(np.int16)

# https://github.com/yash635/Tremolo-Filter/blob/master/tremolo_filter.py
def apply_tremolo(audio_data, sample_rate, depth=0.5, rate=5.0):
    """
    Aplica un efect de tremolo la datele audio.
    Tremolo creeaza o oscilatie periodica a volumului pentru a obtine un efect de vibratie.
    
    Paramaterii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    depth : float
        Profunzimea modulatiei (0.0-1.0), unde 0 = fara efect si 1.0 = modulatie completa
    rate : float
        Frecventa oscilatiei in Hz (cate oscilatii pe secunda)
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu efect de tremolo aplicat
    """
    # Convertim la float pentru procesare
    audio_float = audio_data.astype(np.float32)
    
    # Calculam numarul total de esantioane
    num_samples = len(audio_float)
    
    # Generam un semnal de modulatie sinusoidal
    # Frecventa oscilatiei este determinata de parametrul "rate"
    time = np.arange(num_samples) / sample_rate
    modulation = 1.0 - depth * 0.5 * (1.0 + np.sin(2.0 * np.pi * rate * time))
    
    # Aplicam modulatia la semnalul audio
    tremolo_signal = audio_float * modulation
    
    # Normalizam pentru a preveni clipping
    max_val = np.max(np.abs(tremolo_signal))
    if max_val > 32767:
        tremolo_signal = tremolo_signal * (32767 / max_val)
    
    # Convertim inapoi la int16
    return np.clip(tremolo_signal, -32768, 32767).astype(np.int16)

## 7. Procesarea si aplicarea efectelor

Aceasta sectiune ofera functii de nivel inalt pentru aplicarea efectelor audio pe fisiere complete. Aceste functii servesc drept pod intre interfata utilizator si algoritmii de procesare audio implementati anterior, permitand aplicarea efectelor de o maniera intuitiva si salvarea rezultatelor in fisiere noi.

Functiile principale implementate in aceasta sectiune sunt:
- `process_audio_file()`: Functie generala pentru aplicarea oricarui efect disponibil
- `save_audio_with_effect()`: Interfata intre UI si procesarea propriu-zisa
- `apply_effect_on_wav()`: Functie specializata pentru aplicarea efectelor pe fisiere WAV
- `show_generate_dialog()`: Dialog interactiv pentru generarea semnalelor sinusoidale de test

Sectiunea gestioneaza atat aspectele tehnice (procesarea efectiva a semnalelor) cat si aspectele de interfata utilizator (dialoguri, mesaje de eroare, selectarea fisierelor), oferind o experienta completa si integrata.

In [17]:
# Functii pentru procesarea fisierelor audio si aplicarea efectelor selectate
def process_audio_file(input_file, output_file, effect, effect_params=None):
    """
    Functie generala pentru aplicarea oricarui efect audio pe un fisier.
    Suporta atat WAV cat si MP3.
    
    Paramaterii:
    -----------
    input_file : str
        Calea catre fisierul audio de intrare
    output_file : str
        Calea catre fisierul audio de iesire
    effect : str
        Numele efectului de aplicat
    effect_params : dict, optional
        Parametri suplimentari pentru efect
    """    
    # Extragem datele audio pentru procesare
    try:
        samples, sample_rate = extract_audio_data(input_file)
        # Aplicam efectul selectat
        if effect == "FIR Low Pass":
            cutoff_freq = effect_params.get('cutoff_freq', 1000)
            N = effect_params.get('N', 101)
            processed_audio = fir_low_pass(samples, sample_rate, cutoff_freq, N)
        elif effect == "FIR High Pass":
            cutoff_freq = effect_params.get('cutoff_freq', 1000)
            N = effect_params.get('N', 101)
            processed_audio = fir_high_pass(samples, sample_rate, cutoff_freq, N)
        elif effect == "FIR Band Pass":
            low_cutoff = effect_params.get('low_cutoff', 500)
            high_cutoff = effect_params.get('high_cutoff', 2000)
            N_low = effect_params.get('N_low', 101)
            N_high = effect_params.get('N_high', 101)
            processed_audio = fir_band_pass(samples, sample_rate, low_cutoff, high_cutoff, N_low, N_high)
        elif effect == "FIR Band Reject":
            low_cutoff = effect_params.get('low_cutoff', 500)
            high_cutoff = effect_params.get('high_cutoff', 2000)
            N_low = effect_params.get('N_low', 101)
            N_high = effect_params.get('N_high', 101)
            processed_audio = fir_band_reject(samples, sample_rate, low_cutoff, high_cutoff, N_low, N_high)
        elif effect == "Bass Boost":
            boost_factor = effect_params.get('boost_factor', 2.5)
            cutoff_freq = effect_params.get('cutoff_freq', 150.0)
            processed_audio = apply_bass_boost(samples, sample_rate, boost_factor, cutoff_freq)
        elif effect == "Reverb":
            room_size = effect_params.get('room_size', 0.75)
            damping = effect_params.get('damping', 0.5)
            wet_level = effect_params.get('wet_level', 0.3)
            processed_audio = apply_reverb(samples, sample_rate, room_size, damping, wet_level)
        elif effect == "Echo":
            delays_ms = effect_params.get('delays_ms', [200, 400])
            decays = effect_params.get('decays', [0.5, 0.25])
            processed_audio = apply_echo(samples, sample_rate, delays_ms, decays)
        elif effect == "Speed Up":
            speed_factor = effect_params.get('speed_factor', 1.2)
            preserve_pitch = effect_params.get('preserve_pitch', True)
            processed_audio = apply_speed_up(samples, sample_rate, speed_factor, preserve_pitch)
        elif effect == "Tremolo":
            depth = effect_params.get('depth', 0.5)
            rate = effect_params.get('rate', 5.0)
            processed_audio = apply_tremolo(samples, sample_rate, depth, rate)
        elif effect == "Cancel Noise":
            noise_threshold = effect_params.get('noise_threshold', 0.06)
            processed_audio = apply_noise_cancelling_fft(samples, sample_rate, noise_threshold)
        elif effect == "Equalizer":
            gains = effect_params.get('gains', [1.0, 1.0, 1.0, 1.0, 1.0])
            processed_audio = apply_equalizer(samples, sample_rate, gains)
        elif effect == "Echo + Speed Up":
            delays_ms = effect_params.get('delays_ms', [200, 400])
            decays = effect_params.get('decays', [0.5, 0.25])
            speed_factor = effect_params.get('speed_factor', 1.2)
            preserve_pitch = effect_params.get('preserve_pitch', True)
            
            echo_params = {'delays_ms': delays_ms, 'decays': decays}
            speed_params = {'speed_factor': speed_factor, 'preserve_pitch': preserve_pitch}
            
            processed_audio = apply_echo_speed_combo(samples, sample_rate, echo_params, speed_params)
        else:
            print(f"Efect necunoscut: {effect}")
            
        # Scriem datele procesate in fisierul de iesire
        # Daca output_file are extensie .mp3, salvam ca WAV temporar si apoi convertim la MP3
        output_ext = os.path.splitext(output_file)[1].lower()
        
        wav.write(output_file, sample_rate, processed_audio)
            
    except Exception as e:
        print(f"Eroare la procesarea fisierului audio: {e}")
        
def save_audio_with_effect():
    """
    Functie pentru aplicarea efectului selectat si salvarea rezultatului intr-un nou fisier.
    Daca fisierul este MP3, se afiseaza un mesaj catre utilizator pentru a converti manual fisierul in WAV.
    """
    global current_audio_file, eq_sliders, tremolo_depth_slider, tremolo_rate_slider
    
    if current_audio_file and os.path.exists(current_audio_file):
        # Determinam extensia fisierului
        ext = os.path.splitext(current_audio_file)[1].lower()
        
        # Daca fisierul este MP3, afisam mesaj si intrerupem procesul
        if ext == '.mp3':
            QMessageBox.warning(
                window, 
                "Operatie nesuportata", 
                "Aplicarea efectelor pe fisiere MP3 nu este suportata.\n\n"
                "Pentru a aplica efecte, convertiti intai fisierul la formatul WAV.", 
                QMessageBox.Ok
            )
            return  # Intrerupem functia
            
        # Continuam procesarea pentru fisierele WAV
        effect = effects_combo.currentText()
        
        # Propunem numele fisierului original cu prefix "modified_" pentru salvare
        suggested_name = "modified_" + os.path.basename(current_audio_file)
        
        # Obtinem calea pentru salvare
        output_file, _ = QFileDialog.getSaveFileName(
            window, 
            "Salveaza Fisier Audio Modificat", 
            suggested_name, 
            "Audio Files (*.wav)"  # Restrictionam salvarea doar la WAV
        )
        
        if output_file:
            # Determinam parametrii pentru efect
            effect_params = {}
            
            if effect == "Equalizer" and eq_sliders:
                effect_params['gains'] = [slider.value() / 100.0 for slider in eq_sliders]
                
            elif effect == "Tremolo":
                effect_params['depth'] = tremolo_depth_slider.value() / 100.0
                effect_params['rate'] = tremolo_rate_slider.value()
                
            # Aplicam efectul folosind functia specifica pentru WAV
            apply_effect_on_wav(current_audio_file, output_file, effect, effect_params)
            
            # Afisam un mesaj de succes
            QMessageBox.information(
                window, 
                "Procesare finalizata", 
                f"Fisierul audio a fost procesat si salvat cu succes in:\n{output_file}"
            )
                
            # Actualizam vizualizarile
            update_visualizations()
    else:
        print("Nu exista un fisier audio selectat sau fisierul nu exista")


def apply_effect_on_wav(input_file, output_file, effect, effect_params=None):
    """
    Functie pentru aplicarea efectelor pe fisiere WAV.
    
    Parametrii:
    -----------
    input_file : str
        Calea catre fisierul audio de intrare (WAV)
    output_file : str
        Calea catre fisierul audio de iesire (WAV)
    effect : str
        Numele efectului de aplicat
    effect_params : dict, optional
        Parametri suplimentari pentru efect
    """
    if effect_params is None:
        effect_params = {}
        
    try:
        # Verificam daca fisierul de intrare exista si este WAV
        if not os.path.exists(input_file):
            print(f"Eroare: Fisierul {input_file} nu exista")
            return False
            
        ext = os.path.splitext(input_file)[1].lower()
        if ext != '.wav':
            print(f"Eroare: Fisierul {input_file} nu este in format WAV")
            return False
            
        # Deschidem fisierul audio si citim datele
        with wave.open(input_file, 'rb') as wav_in:
            params = wav_in.getparams()
            num_frames = params.nframes
            framerate = params.framerate
            audio_data = np.frombuffer(wav_in.readframes(num_frames), dtype=np.int16)
            
            # Aplicam efectul selectat
            if effect == "FIR Low Pass":
                cutoff_freq = effect_params.get('cutoff_freq', 1000)
                N = effect_params.get('N', 101)
                processed_audio = fir_low_pass(audio_data, framerate, cutoff_freq, N)
            elif effect == "FIR High Pass":
                cutoff_freq = effect_params.get('cutoff_freq', 1000)
                N = effect_params.get('N', 101)
                processed_audio = fir_high_pass(audio_data, framerate, cutoff_freq, N)
            elif effect == "FIR Band Pass":
                low_cutoff = effect_params.get('low_cutoff', 500)
                high_cutoff = effect_params.get('high_cutoff', 2000)
                N_low = effect_params.get('N_low', 101)
                N_high = effect_params.get('N_high', 101)
                processed_audio = fir_band_pass(audio_data, framerate, low_cutoff, high_cutoff, N_low, N_high)
            elif effect == "FIR Band Reject":
                low_cutoff = effect_params.get('low_cutoff', 500)
                high_cutoff = effect_params.get('high_cutoff', 2000)
                N_low = effect_params.get('N_low', 101)
                N_high = effect_params.get('N_high', 101)
                processed_audio = fir_band_reject(audio_data, framerate, low_cutoff, high_cutoff, N_low, N_high)
            elif effect == "Bass Boost":
                boost_factor = effect_params.get('boost_factor', 2.5)
                cutoff_freq = effect_params.get('cutoff_freq', 150.0)
                processed_audio = apply_bass_boost(audio_data, framerate, boost_factor, cutoff_freq)
            elif effect == "Reverb":
                room_size = effect_params.get('room_size', 0.75)
                damping = effect_params.get('damping', 0.5)
                wet_level = effect_params.get('wet_level', 0.3)
                processed_audio = apply_reverb(audio_data, framerate, room_size, damping, wet_level)
            elif effect == "Echo":
                delays_ms = effect_params.get('delays_ms', [200, 400])
                decays = effect_params.get('decays', [0.5, 0.25])
                processed_audio = apply_echo(audio_data, framerate, delays_ms, decays)
            elif effect == "Speed Up":
                speed_factor = effect_params.get('speed_factor', 1.2)
                preserve_pitch = effect_params.get('preserve_pitch', True)
                processed_audio = apply_speed_up(audio_data, framerate, speed_factor, preserve_pitch)
            elif effect == "Tremolo":
                depth = effect_params.get('depth', 0.5)
                rate = effect_params.get('rate', 5.0)
                processed_audio = apply_tremolo(audio_data, framerate, depth, rate)
            elif effect == "Cancel Noise":
                noise_threshold = effect_params.get('noise_threshold', 0.06)
                processed_audio = apply_noise_cancelling_fft(audio_data, framerate, noise_threshold)
            elif effect == "Equalizer":
                gains = effect_params.get('gains', [1.0, 1.0, 1.0, 1.0, 1.0])
                processed_audio = apply_equalizer(audio_data, framerate, gains)
            elif effect == "Echo + Speed Up":
                delays_ms = effect_params.get('delays_ms', [200, 400])
                decays = effect_params.get('decays', [0.5, 0.25])
                speed_factor = effect_params.get('speed_factor', 1.2)
                preserve_pitch = effect_params.get('preserve_pitch', True)

                echo_params = {'delays_ms': delays_ms, 'decays': decays}
                speed_params = {'speed_factor': speed_factor, 'preserve_pitch': preserve_pitch}

                processed_audio = apply_echo_speed_combo(audio_data, sample_rate, echo_params, speed_params)
            else:
                print(f"Efect necunoscut: {effect}")
                processed_audio = audio_data

            # Scriem datele procesate in fisierul de iesire
            with wave.open(output_file, 'wb') as wav_out:
                wav_out.setparams(params)
                wav_out.writeframes(processed_audio.tobytes())
                
            print(f"Efectul {effect} a fost aplicat cu succes si salvat in {output_file}")
            return True
            
    except Exception as e:
        print(f"Eroare la procesarea fisierului audio: {e}")
        traceback.print_exc()
        return False
        
    return True

# Functia pentru afisarea dialogului de configurare a semnalului
def show_generate_dialog():    
    dialog = QDialog(window)
    dialog.setWindowTitle("Generare Semnal Sinusoidal de Test")
    
    layout = QVBoxLayout()
    form_layout = QFormLayout()
    
    # Configurare frecventa
    freq_spin = QSpinBox()
    freq_spin.setRange(20, 20000)  # Intre 20Hz si 20kHz
    freq_spin.setValue(440)        # Valoare initiala: nota A
    form_layout.addRow("Frecventa (Hz):", freq_spin)
    
    # Configurare durata
    dur_spin = QDoubleSpinBox()
    dur_spin.setRange(0.1, 30.0)   # Intre 0.1 si 30 secunde
    dur_spin.setValue(2.0)         # Valoare initiala: 2 secunde
    dur_spin.setSingleStep(0.1)
    form_layout.addRow("Durata (sec):", dur_spin)
    
    # Configurare amplitudine
    amp_spin = QDoubleSpinBox()
    amp_spin.setRange(0.0, 1.0)    # Intre 0.0 si 1.0
    amp_spin.setValue(0.8)         # Valoare initiala: 0.8
    amp_spin.setSingleStep(0.1)
    form_layout.addRow("Amplitudine (0-1):", amp_spin)
    
    layout.addLayout(form_layout)
    
    # Buton pentru generare
    generate_btn = QPushButton("Genereaza si salveaza")
    
    def on_generate():
        # Genereaza semnalul cu parametrii configurati
        filepath = generate_test_signal(
            freq_spin.value(),
            dur_spin.value(),
            amp_spin.value()
        )
        
        # Adauga fisierul in playlist
        media_content = QMediaContent(QUrl.fromLocalFile(os.path.abspath(filepath)))
        playlist.addMedia(media_content)
        playlist_widget.addItem(os.path.basename(filepath))
        
        dialog.accept()
    
    generate_btn.clicked.connect(on_generate)
    layout.addWidget(generate_btn)
    
    dialog.setLayout(layout)
    dialog.exec_()

## 8. Functii pentru inregistrare

Aceasta sectiune implementeaza functionalitati pentru captarea si procesarea audio in timp real, permitand utilizatorilor sa inregistreze sunet direct din aplicatie. Componentele principale includ:

- **Gestionarea inregistrarii**: Pornirea, oprirea si monitorizarea procesului de inregistrare
- **Manipularea fisierelor**: Crearea si gestionarea fisierelor WAV pentru stocarea inregistrarilor
- **Procesarea in timp real**: Capturarea si conversia datelor audio pentru o calitate optima

Implementarea foloseste biblioteca `sounddevice` pentru accesarea dispozitivelor de inregistrare si un sistem de callback pentru a procesa datele audio pe masura ce sunt captate, asigurand o performanta buna si un consum redus de memorie chiar si pentru inregistrari lungi.

In [18]:
def toggle_recording(record_button):
    """
    Comuta intre starile de inregistrare si oprire.
    Actualizeaza textul butonului si starea de inregistrare.
    
    Parametrii:
    -----------
    record_button : QPushButton
        Referinta catre butonul de inregistrare pentru a-i actualiza textul
    """
    global recording
    if not recording:
        record_button.setText("⏹️ Opreste")
        start_recording()
    else:
        record_button.setText("🎙️ Inregistreaza-te")
        stop_recording()
    recording = not recording
    
def start_recording():
    """
    Initiaza procesul de inregistrare audio.
    Creeaza un fisier WAV nou cu un nume unic, configureaza parametrii audio
    si porneste stream-ul de inregistrare.
    """
    global recording_stream, wav_file, recorded_filename
    fs = 44100  # Frecventa de esantionare standard pentru calitate audio buna
    
    # Sistem de numerotare automata pentru a nu suprascrie inregistrarile existente
    existing_files = [f for f in os.listdir() if f.startswith("recorded_audio_") and f.endswith(".wav")]
    existing_numbers = sorted([int(f.split("_")[-1].split(".")[0]) for f in existing_files if f.split("_")[-1].split(".")[0].isdigit()])
    
    # Gasim primul numar disponibil pentru noua inregistrare
    counter = 1
    while counter in existing_numbers:
        counter += 1
    
    # Generam numele fisierului cu numarul unic
    recorded_filename = f"recorded_audio_{counter}.wav"
    
    # Initializare fisier WAV pentru scriere
    wav_file = wave.open(recorded_filename, 'wb')
    # Configurare parametri audio pentru fisierul WAV:
    wav_file.setnchannels(1)       # Mono (1 canal)
    wav_file.setsampwidth(2)       # 2 bytes per sample (16 bit)
    wav_file.setframerate(fs)      # Frecventa de esantionare setata anterior
    
    # Initializare stream de inregistrare cu callback pentru procesarea datelor in timp real
    recording_stream = sd.InputStream(samplerate=fs, channels=1, callback=callback)
    recording_stream.start()

def stop_recording():
    """
    Opreste procesul de inregistrare audio.
    Inchide stream-ul de inregistrare si fisierul WAV, apoi adauga
    inregistrarea finalizata in playlist pentru a putea fi redata.
    Actualizeaza variabila current_audio_file pentru a permite aplicarea efectelor.
    """
    global recording_stream, wav_file, recorded_filename, current_audio_file
    # Oprim si inchidem stream-ul de inregistrare
    if recording_stream:
        recording_stream.stop()
        recording_stream.close()
    # Inchidem fisierul WAV pentru a finaliza scrierea
    if wav_file:
        wav_file.close()

    # Obtinem calea absoluta catre fisierul inregistrat
    recording_path = os.path.abspath(recorded_filename)
    
    # Actualizam current_audio_file pentru a putea aplica efecte pe inregistrare
    current_audio_file = recording_path
    
    # Adaugam fisierul inregistrat in playlist si in interfata vizuala
    playlist.addMedia(QMediaContent(QUrl.fromLocalFile(recording_path)))
    playlist_widget.addItem(os.path.basename(recorded_filename))
    
    # Actualizam vizualizarile
    update_visualizations()

def callback(indata, frames, time, status):
    """
    Functie callback apelata de sounddevice pentru fiecare bloc de date audio capturat.
    Proceseaza datele primite si le scrie in fisierul WAV.
    
    Parametrii:
    -----------
    indata : numpy.ndarray
        Buffer cu datele audio capturate (valori float intre -1.0 si 1.0)
    frames : int
        Numarul de frame-uri in buffer
    time : CData
        Informatii despre timestamp
    status : CallbackFlags
        Indicator de stare sau eroare pentru callback
    """
    # Verificam daca exista erori in procesul de captare
    if status:
        print(status)
    
    # Convertim valorile float din indata (-1.0 pana la 1.0) in valori int16 (-32768 pana la 32767)
    # Aceasta este conversia standard pentru audio PCM pe 16 biti
    int_data = np.int16(indata * 32767)
    
    # Scriem datele convertite in fisierul WAV
    wav_file.writeframes(int_data.tobytes())

## 9. Initializare UI

Aceasta sectiune creeaza si configureaza interfata grafica a aplicatiei, definind toate componentele vizuale si conexiunile dintre acestea. Interfata este organizata in tab-uri pentru o navigare usoara si o structura clara, oferind utilizatorului toate controalele necesare pentru interactiunea cu fisierele audio.

Componentele principale ale interfetei includ:
- Player-ul audio cu playlist si controale de redare
- Controale pentru efecte si vizualizari audio
- Functionalitati de inregistrare si generare de semnale
- Vizualizari avansate (spectrograma si spectrul de frecventa)

Functia `init_ui()` reprezinta punctul central de configurare a interfetei, creand fereastra principala, distribuind componente in layout-uri potrivite si stabilind conexiunile intre semnale si sloturi pentru a asigura interactivitatea.

In [19]:
# Aceasta sectiune creeaza interfata grafica si initializeaza componentele principale.
def init_ui():
    """
    Creeaza interfata grafica si initializeaza componentele principale.
    Adauga suport pentru vizualizarea spectrogramei si informatiilor audio.
    """
    global window, media_player, playlist, playlist_widget, speed_slider, volume_slider
    global progress_slider, effects_combo, eq_sliders, tremolo_depth_slider, tremolo_rate_slider
    global spectrogram_canvas, frequency_spectrum_canvas, status_label
    
    # Creare fereastra principala
    window = QMainWindow()
    window.setWindowTitle("Audio Player Pro 🔥")
    window.setGeometry(100, 100, 1000, 800)  # Fereastra mai mare pentru a acomoda vizualizarile
    
    # Initializare componente media
    media_player = QMediaPlayer()
    playlist = QMediaPlaylist()
    media_player.setPlaylist(playlist)
    
    # Conectam semnalul de eroare pentru a putea depana probleme
    media_player.error.connect(lambda: print(f"Eroare media player: {media_player.errorString()}"))
    media_player.mediaStatusChanged.connect(lambda status: print(f"Status media: {status}"))
    media_player.stateChanged.connect(lambda state: print(f"State changed: {state}"))
    
    # Widget-ul principal cu tab-uri pentru organizarea interfetei
    central_widget = QTabWidget()
    
    # ===== Tab-ul principal pentru player =====
    player_tab = QWidget()
    main_layout = QVBoxLayout()
    
    # Adaugam un label pentru status care va afisa informatii despre redare
    status_label = QLabel("Gata pentru redare")
    status_label.setStyleSheet("font-weight: bold; color: #555;")
    main_layout.addWidget(status_label)
    
    # Lista cu melodii
    playlist_widget = QListWidget()
    playlist_widget.itemDoubleClicked.connect(play_selected_song)  # Activare la dublu click
    main_layout.addWidget(QLabel("🎵 Playlist:"))
    main_layout.addWidget(playlist_widget)
    
    # Butoane pentru control
    controls_layout = QHBoxLayout()
    
    # Definirea butoanelor si conectarea lor la functiile corespunzatoare
    buttons = {
        "📂 Incarca": open_audio_file,
        "▶ Play": toggle_play,
        "■ Stop": stop_audio,
        "🔀 Shuffle": toggle_shuffle,
        "🔁 Repeat": toggle_repeat,
        "🗑️ Sterge": remove_selected_song,
        "💾 Aplica Efect": save_audio_with_effect,
        "ℹ️ Info": show_audio_info,
        "🔊 Compara Canale": compare_stereo_channels
    }
    
    # Creare butoane
    for text, func in buttons.items():
        btn = QPushButton(text)
        btn.clicked.connect(func)
        controls_layout.addWidget(btn)

    # Adauga un buton pentru generarea de semnale de test
    generate_button = QPushButton("🎵 Genereaza Semnal Sinusoidal")
    generate_button.clicked.connect(lambda: show_generate_dialog())
    controls_layout.addWidget(generate_button)

    # Buton de inregistrare
    record_button = QPushButton("🎙️ Inregistreaza-te")
    record_button.clicked.connect(lambda: toggle_recording(record_button))
    controls_layout.addWidget(record_button)
    
    main_layout.addLayout(controls_layout)
    
    # Slider pentru viteza de redare (50% - 200%)
    speed_slider = QSlider(Qt.Horizontal)
    speed_slider.setRange(50, 200)
    speed_slider.setValue(100)  # Valoare implicita: 100% = viteza normala
    speed_slider.valueChanged.connect(change_speed)
    main_layout.addWidget(QLabel("⏩ Viteza Redare:"))
    main_layout.addWidget(speed_slider)
    
    # Slider pentru volum (0% - 100%)
    volume_slider = QSlider(Qt.Horizontal)
    volume_slider.setRange(0, 100)
    volume_slider.setValue(50)  # Valoare implicita: 50%
    volume_slider.sliderMoved.connect(change_volume)
    main_layout.addWidget(QLabel("🔊 Volum:"))
    main_layout.addWidget(volume_slider)
    
    # Dropdown pentru efecte audio
    effects_combo = QComboBox()

    """
    TO-DO 3: adaugare efecte
    """
    
    effects_combo.addItems([
        "None",
        "FIR Low Pass",
        "FIR High Pass",
        "FIR Band Pass",
        "FIR Band Reject",
        "Bass Boost",
        "Reverb",
        "Echo",
        "Speed Up",
        "Tremolo",
        "Cancel Noise",
        "Equalizer",
        "Echo + Speed Up"
    ])
    main_layout.addWidget(QLabel("🎚️ Efecte Audio:"))
    main_layout.addWidget(effects_combo)
    
    # Adaugare layout pentru equalizer si tremolo
    effects_params_layout = QHBoxLayout()
    
    # Container pentru slider-ele equalizer-ului
    eq_container = QWidget()
    eq_container.setVisible(False)  # Initial ascuns
    eq_sliders_layout = QHBoxLayout()
    eq_container.setLayout(eq_sliders_layout)
    
    # Cream sliderele si etichetele pentru cele 5 benzi
    eq_sliders = []
    eq_labels = ["Bass", "Low-Mid", "Mid", "High-Mid", "Treble"]
    
    for label in eq_labels:
        slider_layout = QVBoxLayout()
        
        # Adaugam eticheta
        slider_layout.addWidget(QLabel(label))
        
        # Cream slider-ul vertical
        slider = QSlider(Qt.Vertical)
        slider.setRange(10, 300)  # 10% - 300% (0.1 - 3.0)
        slider.setValue(100)      # Valoare implicita: 100% = 1.0 (neutru)
        slider.setFixedHeight(100)
        
        slider_layout.addWidget(slider)
        eq_sliders.append(slider)
        eq_sliders_layout.addLayout(slider_layout)
    
    # Adaugam containerul de slidere in layout-ul principal
    effects_params_layout.addWidget(eq_container)
    
    # Container pentru parametrii Tremolo
    tremolo_container = QWidget()
    tremolo_container.setVisible(False)  # Initial ascuns
    tremolo_layout = QHBoxLayout()
    tremolo_container.setLayout(tremolo_layout)

    # Slider pentru profunzimea Tremolo
    tremolo_depth_layout = QVBoxLayout()
    tremolo_depth_layout.addWidget(QLabel("Profunzime"))
    tremolo_depth_slider = QSlider(Qt.Vertical)
    tremolo_depth_slider.setRange(0, 100)  # 0-100% (0.0-1.0)
    tremolo_depth_slider.setValue(50)      # Valoare implicita: 50% = 0.5
    tremolo_depth_slider.setFixedHeight(100)
    tremolo_depth_layout.addWidget(tremolo_depth_slider)

    # Slider pentru rata Tremolo
    tremolo_rate_layout = QVBoxLayout()
    tremolo_rate_layout.addWidget(QLabel("Rata (Hz)"))
    tremolo_rate_slider = QSlider(Qt.Vertical)
    tremolo_rate_slider.setRange(1, 20)    # 1-20 Hz
    tremolo_rate_slider.setValue(5)        # Valoare implicita: 5 Hz
    tremolo_rate_slider.setFixedHeight(100)
    tremolo_rate_layout.addWidget(tremolo_rate_slider)

    # Adaugam componentele in container
    tremolo_layout.addLayout(tremolo_depth_layout)
    tremolo_layout.addLayout(tremolo_rate_layout)

    # Adaugam containerul in layout-ul principal
    effects_params_layout.addWidget(tremolo_container)
    
    main_layout.addLayout(effects_params_layout)

    # Functie pentru a afisa/ascunde containerele de parametri in functie de efectul selectat
    def toggle_effect_controls(index):
        current_effect = effects_combo.currentText()
        eq_container.setVisible(current_effect == "Equalizer")
        tremolo_container.setVisible(current_effect == "Tremolo")

    effects_combo.currentIndexChanged.connect(toggle_effect_controls)
    
    # Slider pentru progresul redarii (0% - 100%)
    progress_slider = QSlider(Qt.Horizontal)
    progress_slider.setRange(0, 100)
    progress_slider.sliderMoved.connect(set_position)
    main_layout.addWidget(QLabel("📈 Progres:"))
    main_layout.addWidget(progress_slider)
    
    # Adaugam un label pentru afisarea starii curente si erorilor
    current_status_label = QLabel("Stare: Gata")
    main_layout.addWidget(current_status_label)
    
    # Actualizeaza label-ul cand se schimba starea player-ului
    def update_status_label(state):
        if state == QMediaPlayer.PlayingState:
            current_status_label.setText("Stare: Redare")
        elif state == QMediaPlayer.PausedState:
            current_status_label.setText("Stare: Pauza")
        elif state == QMediaPlayer.StoppedState:
            current_status_label.setText("Stare: Oprit")
    
    media_player.stateChanged.connect(update_status_label)
    
    # Functie pentru afisarea erorilor
    def show_error(error):
        if error != QMediaPlayer.NoError:
            current_status_label.setText(f"Eroare: {media_player.errorString()}")
    
    media_player.error.connect(show_error)
    
    player_tab.setLayout(main_layout)
    
    # ===== Tab pentru spectrograma =====
    spectrogram_tab = QWidget()
    spectrogram_layout = QVBoxLayout()
    
    # Adaugam titlul
    spectrogram_layout.addWidget(QLabel("Spectrograma:"))
    
    # Cream canvas-ul pentru spectrograma
    spectrogram_canvas = SpectrogramCanvas()
    spectrogram_layout.addWidget(spectrogram_canvas)
    
    # Adaugam explicatii
    spectro_info = QLabel(
        "Spectrograma arata distributia frecventelor in timp. "
        "Culorile mai intense reprezinta amplitudini mai mari. "
        "Frecventele joase (bass) sunt in partea de jos, iar cele inalte (treble) in partea de sus."
    )
    spectro_info.setWordWrap(True)
    spectrogram_layout.addWidget(spectro_info)
    
    spectrogram_tab.setLayout(spectrogram_layout)

    # ===== Tab pentru spectru de frecventa =====
    frequency_spectrum_tab = QWidget()
    frequency_spectrum_layout = QVBoxLayout()
    
    # Adaugam titlul
    frequency_spectrum_layout.addWidget(QLabel("Spectru de Frecventa:"))

    # IMPORTANT: Aici este problema - trebuie sa initializam corespunzator frequency_spectrum_canvas
    frequency_spectrum_canvas = FrequencySpectrumCanvas()
    frequency_spectrum_layout.addWidget(frequency_spectrum_canvas)

    # Adaugam explicatii
    spectrum_info = QLabel(
        "Spectrul de frecventa arata amplitudinea semnalului audio in functie de frecventa. "
        "Frecventele joase (bass) sunt in partea stanga, iar cele inalte (treble) in partea dreapta."
    )
    spectrum_info.setWordWrap(True)
    frequency_spectrum_layout.addWidget(spectrum_info)

    frequency_spectrum_tab.setLayout(frequency_spectrum_layout)
    
    # Adaugam un buton pentru actualizarea fortata a vizualizarilor in ambele tab-uri de vizualizare
    update_button = QPushButton("🔄 Actualizeaza Vizualizari")
    
    def force_update_visualizations():
        """
        Forteaza reincarcarea si afisarea vizualizarilor pentru fisierul audio curent.
        """
        global current_audio_file, spectrogram_canvas, frequency_spectrum_canvas
        
        print("Fortare actualizare vizualizari...")
        
        if current_audio_file and os.path.exists(current_audio_file):
            print(f"Incercare fortata de actualizare pentru fisierul: {current_audio_file}")
            
            # Verificam daca canvas-urile sunt initializate
            if spectrogram_canvas is None:
                print("EROARE: spectrogram_canvas nu este initializat!")
                return
                
            if frequency_spectrum_canvas is None:
                print("EROARE: frequency_spectrum_canvas nu este initializat!")
                return
                
            try:
                # Extragem datele audio direct pentru vizualizare
                samples, sample_rate = extract_audio_data(current_audio_file, keep_stereo=False)
                
                if samples is None or sample_rate is None:
                    print("EROARE: Nu s-au putut extrage date audio pentru vizualizare")
                    return
                    
                print(f"Actualizare fortata cu {len(samples)} samples la {sample_rate}Hz")
                
                # Actualizam direct spectrograma
                spectrogram_canvas.plot_spectrogram(samples, sample_rate)
                
                # Actualizam direct spectrul de frecventa
                frequency_spectrum_canvas.plot_frequency_spectrum(samples, sample_rate)
                
                print("Actualizare fortata finalizata cu succes")
            except Exception as e:
                print(f"Eroare la actualizarea fortata: {e}")

                traceback.print_exc()
        else:
            if not current_audio_file:
                print("Nu exista un fisier audio curent pentru actualizare fortata")
            else:
                print(f"Fisierul {current_audio_file} nu exista")
    
    update_button.clicked.connect(force_update_visualizations)
    spectrogram_layout.addWidget(update_button)
    
    # Adaugam acelasi buton si in tab-ul pentru spectrul de frecventa
    update_button2 = QPushButton("🔄 Actualizeaza Vizualizari")
    update_button2.clicked.connect(force_update_visualizations)
    frequency_spectrum_layout.addWidget(update_button2)
    
    # Adaugam tab-urile in widget-ul principal
    central_widget.addTab(player_tab, "Player")
    central_widget.addTab(spectrogram_tab, "Spectrograma")
    central_widget.addTab(frequency_spectrum_tab, "Spectru de Frecventa")
    
    # Conectare semnal pentru actualizarea progresului
    media_player.positionChanged.connect(update_progress)
    
    # Setare widget central
    window.setCentralWidget(central_widget)
    
    # Adaugam o verificare pentru a confirma ca toate variabilele globale importante sunt initializate
    print("Verificare initializare variabile importante:")
    print(f"spectrogram_canvas initializat: {spectrogram_canvas is not None}")
    print(f"frequency_spectrum_canvas initializat: {frequency_spectrum_canvas is not None}")
    print(f"current_audio_file initializat: {current_audio_file is not None}")
    
    # Conectam semnalul de schimbare a tab-ului pentru a actualiza vizualizarile cand se schimba tab-ul
    def tab_changed(index):
        if index == 1 or index == 2:  # Daca s-a selectat tab-ul de spectrograma sau spectru
            print(f"Tab schimbat la {index}, actualizez vizualizarile")
            force_update_visualizations()
    
    central_widget.currentChanged.connect(tab_changed)

## 10. Exerciții (TO-DO)

### 10.1. Generator de semnale de test
Acest exercitiu consta in implementarea unui generator de semnale sinusoidale pentru testarea aplicatiei. Un semnal sinusoidal este caracterizat de o forma de unda oscilanta in forma de sinus.

### Indicații
* Numărul de eșantioane este egal cu produsul dintre durata semnalului și rata de eșantionare.
* Se poate calcula timpul pentru eșantionare utilizând *linspace*.
* Pentru a genera un semnal sinusoidal de amplitudine 1, se poate folosi funția np.sin().
* Normalizarea semnalului: semnal = semnal * 32768, unde semnalul nostru este de forma A * sin(2 * pi * w * t).
* La final, se convertește la formatul audio, utilizănd np.clip(), ca în exemplele de mai sus.


In [20]:
# ====== TO-DO 1: Implementeaza un generator de semnale sinusoidale de test ======
#
# Creeaza o functie care:
# 1. Genereaza semnale de test (sinusoidal)
# 2. Permite configurarea frecventei, duratei si amplitudinii
# 3. Salveaza semnalele generate ca fisiere WAV pentru testare

def generate_test_signal(frequency, duration, amplitude=0.8, sample_rate=44100, output_dir="test_signals"):
   """
   Genereaza un semnal de test.
   
   Parametri:
   ----------
   frequency : float
       Frecventa semnalului in Hz
   duration : float
       Durata semnalului in secunde
   amplitude : float
       Amplitudinea semnalului (0.0-1.0)
   sample_rate : int
       Rata de esantionare
   output_dir : str
       Directorul in care se salveaza fisierul
       
   Returneaza:
   ----------
   numpy.ndarray
       Semnalul generat
   """
   # TODO: Calculeaza numarul de esantioane in functie de durata
   
   # TODO: Genereaza timpul pentru esantioane
   
   # TODO: Implementeaza generarea de semnal sinusoidal (sin)
   
   # TODO: Normalizeaza semnalul la amplitudinea specificata
   
   # TODO: Converteste la formatul de audio (int16)

   # Creeaza directorul de iesire daca nu exista
   if not os.path.exists(output_dir):
       os.makedirs(output_dir)
   
   # Creeaza numele fisierului
   filename = f"sine_{frequency}Hz_{duration}s.wav"
   filepath = os.path.join(output_dir, filename)
   
   # Salveaza fisierul WAV
   wav.write(filepath, sample_rate, signal)
   
   print(f"Semnal sinusoidal generat si salvat in {filepath}")
   return filepath

### 10.2. Demonstrație unde opuse

Acest exercitiu ilustreaza un concept fundamental in acustica: anularea undelor sonore prin interferenta destructiva. Atunci cand doua unde cu aceeasi amplitudine dar faze opuse se intalnesc, ele se anuleaza reciproc rezultand in lipsa sunetului.

#### Principiul fizic
Cand o unda este inversata (faza opusa), amplitudinea sa in fiecare punct devine opusul amplitudinii originale. La combinarea celor doua unde, valorile se vor anula reciproc (adunarea unui numar cu opusul sau da zero), demonstrand principiul de functionare al tehnologiilor de anulare a zgomotului.

#### Implementarea practica
1. **Citirea fisierului audio**
  - Vom folosi functia `wav.read()` care returneaza rata de esantionare si datele audio
  - Aceasta functie extrage samples-urile audio din fisierul WAV ca un array NumPy

2. **Crearea versiunii opuse**
  - Pentru a crea o versiune opusa, este suficient sa inmultim toate valorile cu -1
  - Acest lucru inverseaza faza semnalului, transformand vaile in varfuri si invers

3. **Combinarea undelor**
  - Adunarea celor doua semnale (original + inversat) ar trebui sa rezulte in anulare
  - In practica, rezultatul nu va fi exact zero din cauza limitarilor digitale si rotunjirilor

4. **Salvarea rezultatelor**
  - Vom salva trei fisiere WAV: originalul, versiunea inversata, si rezultatul combinarii
  - Aceste fisiere pot fi comparate auditiv pentru a observa efectul

5. **Vizualizare grafica**
  - Functia `create_comparison_graph()` va genera un grafic cu trei subploturi
  - Acest grafic ofera o reprezentare vizuala a undelor si a efectului anularii

#### Aplicatii practice
Acest principiu este folosit in:
- Casti cu anulare activa a zgomotului
- Sisteme de control al zgomotului in spatii industriale
- Testarea caracteristicilor acustice ale materialelor

Implementarea acestui exercitiu va ajuta la intelegerea modului in care undele sonore interactioneaza si cum pot fi manipulate matematic in domeniul digital.

In [21]:
# ====== TO-DO 2: Functie pentru demonstrarea anularii undelor sonore ======
#
# Creeaza o functie care:
# 1. Foloseste fisierul audio dat ca parametru
# 2. Creeaza o versiune "opus" a undei (Indiciu: inmulteste cu -1)
# 3. Combina cele doua unde prin adunare
# 4. Salveaza toate fisierele si afiseaza un grafic comparativ

def create_and_combine_opposite_waves(input_file, output_folder="wave_demonstration"):
   """
   Demonstreaza anularea undelor sonore prin combinarea unui sunet cu versiunea sa opusa.
   
   Parametrii:
   -----------
   input_file : str
       Calea catre fisierul audio de intrare
   output_folder : str
       Directorul in care se vor salva fisierele rezultate
   
   Returneaza:
   -----------
   bool
       True daca procesul s-a finalizat cu succes, False altfel
   """
   try:
       # TODO: Implementeaza citirea fisierului audio folosind wav.read 
       
       # TODO: Creeaza versiunea opusa prin inmultirea cu -1 a sample-urilor.
       
       # TODO: Combina cele doua versiuni prin adunare.
       
       # Extragem numele fisierului fara extensie
       base_name = os.path.splitext(os.path.basename(input_file))[0]
       
       # Cream directorul de iesire daca nu exista
       if not os.path.exists(output_folder):
           os.makedirs(output_folder)
       
       # Definim caile pentru fisierele de iesire
       original_path = os.path.join(output_folder, f"{base_name}_original.wav")
       inverted_path = os.path.join(output_folder, f"{base_name}_inverted.wav")
       combined_path = os.path.join(output_folder, f"{base_name}_combined.wav")
       graph_path = os.path.join(output_folder, f"{base_name}_comparison.png")
       
       # TODO: Salveaza fiecare versiune ca fisier WAV, utilizand wav.write()
       
       # Cream si salvam un grafic comparativ al undelor
       create_comparison_graph(samples, inverted_samples, combined_samples, graph_path)
       
       print(f"Fisiere generate cu succes in directorul: {output_folder}")
       return True
       
   except Exception as e:
       print(f"Eroare la procesarea fisierului audio: {e}")
       traceback.print_exc()
       return False

def create_comparison_graph(original, inverted, combined, output_path, max_samples=1000):
   """
   Creeaza un grafic comparativ al celor trei forme de unda.
   
   Parametrii:
   -----------
   original : numpy.ndarray
       Datele audio originale
   inverted : numpy.ndarray
       Datele audio inversate
   combined : numpy.ndarray
       Datele audio combinate
   output_path : str
       Calea catre fisierul de iesire pentru grafic
   max_samples : int
       Numarul maxim de sample-uri de afisat pentru claritate
   """
   # Limitam numarul de sample-uri pentru claritate
   if len(original) > max_samples:
       start_idx = len(original) // 2 - max_samples // 2  # Luam din mijlocul semnalului
       samples_to_show = slice(start_idx, start_idx + max_samples)
   else:
       samples_to_show = slice(0, len(original))
   
   # Cream figura si subploturile
   plt.figure(figsize=(10, 8))
   
   # Subplot pentru unda originala
   plt.subplot(3, 1, 1)
   plt.plot(original[samples_to_show])
   plt.title('Semnal Original')
   plt.grid(True)
   
   # Subplot pentru unda inversata
   plt.subplot(3, 1, 2)
   plt.plot(inverted[samples_to_show], 'r')
   plt.title('Semnal Inversat')
   plt.grid(True)
   
   # Subplot pentru unda combinata
   plt.subplot(3, 1, 3)
   plt.plot(combined[samples_to_show], 'g')
   plt.title('Semnal Combinat (Anulat)')
   plt.grid(True)
   
   plt.tight_layout()
   plt.savefig(output_path)
   plt.close()

In [22]:
create_and_combine_opposite_waves("eminem-30s.wav") # introduceti aici numele fisierului cu extensia .wav

Eroare la procesarea fisierului audio: name 'samples' is not defined


Traceback (most recent call last):
  File "C:\Users\plete\AppData\Local\Temp\ipykernel_17304\1566200962.py", line 48, in create_and_combine_opposite_waves
    create_comparison_graph(samples, inverted_samples, combined_samples, graph_path)
NameError: name 'samples' is not defined


False

### 10.3. Combinare efecte audio

Creeaza o functie care:
1. Primeste un semnal audio ca input.
2. Aplica efectele **Echo** si **Speed-Up**, folosind functiile existente `apply_echo` si `apply_speed_up`.
3. Experimenteaza cu ordinea aplicarii efectelor pentru a observa cum influenteaza rezultatul final.

---

### Explicatii

#### **Initializarea parametrilor**  
Daca `echo_params` sau `speed_params` sunt `None`, trebuie sa fie initializate cu valori implicite.

#### **Aplicarea efectului Echo**  
Se foloseste functia `apply_echo` pentru a adauga un efect de ecou asupra semnalului audio.

#### **Aplicarea efectului Speed-Up**  
Se aplica `apply_speed_up` pentru a accelera redarea sunetului.

#### **Normalizarea semnalului**  
Dupa aplicarea efectelor, trebuie sa ne asiguram ca volumul rezultat nu produce distorsiuni.

---

### **Observatie**  
Ordinea aplicarii efectelor va influenta rezultatul final:

- **Echo → Speed-Up**: Ecoul poate deveni mai rapid si mai strans.  
- **Speed-Up → Echo**: Ecoul poate fi mai amplificat si mai dispersat.


In [23]:
# ====== TO-DO 3: Combina efectele Echo si Speed-Up ======
#
# Creeaza o functie care:
# 1. Ia un semnal audio ca input
# 2. Foloseste efectele apply_echo si apply_speed_up existente pentru a crea un efect combinat
# 3. Experimenteaza cu ordinea de aplicare si cum afecteaza rezultatul final

def apply_echo_speed_combo(audio_data, sample_rate, echo_params=None, speed_params=None):
    """
    Combina efectele Echo si Speed-Up pentru a crea un efect audio complex.
    
    Parametrii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    echo_params : dict, optional
        Parametri pentru efectul echo 
    speed_params : dict, optional
        Parametri pentru efectul speed-up
    
    Returneaza:
    --------
    numpy.ndarray
        Datele audio cu efectul combinat aplicat
    """
    # TODO: Initializeaza parametrii implicit daca sunt None
    
    # TODO: Aplica efectul de echo folosind functia apply_echo
    
    # TODO: Aplica efectul de speed-up folosind functia apply_speed_up
    
    # TODO: Normalizeaza rezultatul pentru a preveni distorsiunea
    
    pass


### 10.4 Filtre audio
Creeaza o functie care:
1. Foloseste functia existenta pentru filtrul trece-jos (`fir_low_pass`).
2. Transforma filtrul trece-jos intr-un filtru trece-sus prin inversiune spectrala.
3. Permite configurarea frecventei de taiere si a ordinului filtrului.
4. Testeaza si demonstreaza rezultatul prin aplicarea filtrului unui fisier `.wav`, prin intermediul interfetei grafice.

---

### Explicatii

#### **Normalizarea frecventei de taiere**  
Frecventa de taiere trebuie exprimata ca o proportie din jumatate din rata de esantionare (`cutoff_freq / (sample_rate / 2)`).

#### **Calcularea filtrului sinc**  
Filtrul trece-jos este generat folosind o functie sinc, la fel ca in `fir_low_pass`.

#### **Aplicarea ferestrei Hamming**  
Aceasta ajuta la reducerea efectului de ripple in frecvente.

#### **Normalizarea castigului**  
Coeficientii filtrului trebuie ajustati pentru a asigura un raspuns plat in frecventa dorita.

#### **Inversiunea spectrala**  
Transformarea filtrului trece-jos in trece-sus se face astfel:  

- Se inmultesc toti coeficientii filtrului cu `-1`.  
- Se adauga un impuls delta (`1`) in centrul filtrului.  

#### **Aplicarea filtrului prin convolutie**  
Filtrul este aplicat asupra semnalului audio prin operatia de convolutie.

#### **Normalizarea semnalului**  
Dupa filtrare, volumul trebuie ajustat pentru a preveni distorsiuni si clipping.

---

### **Observatie**  
Acest filtru permite eliminarea componentelor de frecventa joasa si pastrarea celor inalte, fiind util in diverse aplicatii audio, precum imbunatatirea claritatii vocale sau eliminarea zgomotelor de fond joase.


In [24]:
def fir_high_pass(audio_data, sample_rate, cutoff_freq, N, outputType=np.int16):
    """
    Aplica un filtru trece-sus FIR la datele audio.
    
    Parametrii:
    -----------
    audio_data : numpy.ndarray
        Datele audio de intrare
    sample_rate : int
        Rata de esantionare a semnalului audio
    cutoff_freq : float
        Frecventa de taiere in Hz
    N : int
        Ordinul filtrului (numarul de coeficienti)
    outputType : numpy.dtype, optional
        Tipul de date pentru iesire (default: np.int16)
        
    Returneaza:
    --------
    numpy.ndarray
        Datele audio filtrate
    """
    
    # TODO: Normalizeaza frecventa de taiere
    
    # TODO: Calculeaza filtrul sinc (identic cu fir_low_pass)
    
    # TODO: Aplica fereastra Hamming pentru reducerea ripple
    
    # TODO: Normalizeaza pentru a obtine un castig unitar
    
    # TODO: Realizeaza inversiunea spectrala:
    #       1. Inverseaza toti coeficientii (inmulteste cu -1)
    #       2. Adauga un impuls delta in centrul filtrului (adauga 1)
    
    # TODO: Aplica filtrul prin convolutie
    
    # TODO: Normalizeaza semnalul pentru a preveni clipping
    
    pass

### 10.5 Generator de Cod Morse Audio

#### **Conversia textului in cod Morse**
- Se defineste un dictionar cu echivalentele dintre litere/cifre si codul Morse.
- Textul este convertit la uppercase pentru a asigura compatibilitatea.
- Fiecare caracter este inlocuit cu secventa corespunzatoare din codul Morse.
- Se adauga spatii intre simboluri pentru claritate.

#### **Generarea semnalului audio**
- Se defineste durata standard pentru punct (`.`) si linie (`-`).
- Se genereaza un semnal sinusoidal pentru fiecare punct si linie.
- Se adauga pauze intre simboluri, litere si cuvinte.

#### **Normalizarea semnalului audio**
- Se ajusteaza amplitudinea semnalului pentru a preveni distorsiunile.
- Se salveaza semnalul in format `.wav` folosind un sample rate de `44100 Hz`.

---

### **Structura codului**

#### **`text_to_morse(text)`**  
- Converteste textul in cod Morse utilizand un dictionar predefinit.

#### **`morse_to_audio(text, output_file, dot_duration=0.1, frequency=800)`**  
- Converteste textul in cod Morse.
- Genereaza un fisier audio `.wav` cu frecventa si durata specificate.
- Utilizeaza operatii matematice pentru a crea tonuri sinusoidale.
- Normalizeaza si salveaza fisierul audio.

---

#### **Observatii**
- Codul Morse este util pentru comunicatii in conditii de zgomot ridicat.
- Frecventa tonului poate fi ajustata in functie de preferinte.
- Se pot experimenta diverse durate pentru punct si linie pentru a imbunatati claritatea semnalului.

In [25]:
# ====== TO-DO 5: Generator de Cod Morse Audio Simplificat ======
#
# Creeaza o functionalitate care:
# 1. Convertește un text in cod Morse
# 2. Genereaza un fisier audio WAV cu semnalele corespunzatoare
#
# Utilizare simpla: 
# morse_to_audio("TEXT DE CONVERTIT", "nume_fisier_output.wav")

# https://ocw.cs.pub.ro/courses/pm/lab/lab1-2023
# https://ocw.cs.pub.ro/courses/pm/lab/lab2-2023
def text_to_morse(text):
    """
    Convertește un text in cod Morse.
    
    Parametri:
    ----------
    text : str
        Textul care va fi convertit in cod Morse
        
    Returneaza:
    ----------
    str
        Reprezentarea in cod Morse a textului
    """
    # Dictionarul de conversie pentru cod Morse
    morse_dict = {
        'A': '.-', 'B': '-...', 'C': '-.-.', 'D': '-..', 'E': '.', 'F': '..-.', 
        'G': '--.', 'H': '....', 'I': '..', 'J': '.---', 'K': '-.-', 'L': '.-..', 
        'M': '--', 'N': '-.', 'O': '---', 'P': '.--.', 'Q': '--.-', 'R': '.-.', 
        'S': '...', 'T': '-', 'U': '..-', 'V': '...-', 'W': '.--', 'X': '-..-', 
        'Y': '-.--', 'Z': '--..', '0': '-----', '1': '.----', '2': '..---', 
        '3': '...--', '4': '....-', '5': '.....', '6': '-....', '7': '--...', 
        '8': '---..', '9': '----.', ' ': '/'
    }
    
    # TODO: Convertește textul la uppercase

    # TODO: Convertește fiecare caracter in cod Morse

    # TODO: Combină caracterele cu spatii intre ele


def morse_to_audio(text, output_file, dot_duration=0.1, frequency=800):
    """
    Convertește text in cod Morse și apoi in audio.
    
    Parametri:
    ----------
    text : str
        Textul care va fi convertit
    output_file : str
        Numele fisierului WAV de iesire
    dot_duration : float, optional
        Durata unui punct in secunde (default: 0.1)
    frequency : int, optional
        Frecventa tonului in Hz (default: 800)
        
    Returneaza:
    ----------
    str
        Codul Morse generat
    """
    # Convertim textul in cod Morse
    morse_code = text_to_morse(text)
    print(f"Textul '{text}' a fost convertit in codul Morse: {morse_code}")
    
    # Parametri audio
    sample_rate = 44100  # Hz
    
    # Duratele elementelor Morse (in secunde)
    dash_duration = 3 * dot_duration
    element_gap = dot_duration  # pauza intre puncte si linii in acelasi caracter
    char_gap = 3 * dot_duration  # pauza intre caractere
    
    # Generam semnalul audio
    signal = []
    
    for i, char in enumerate(morse_code):
        if char == '.':
            # Adaugam un punct (ton scurt)
            t = np.linspace(0, dot_duration, int(dot_duration * sample_rate), False)
            tone = np.sin(2 * np.pi * frequency * t)
            signal.extend(tone)
            
            # Adaugam o pauza dupa element (exceptand ultimul)
            if i < len(morse_code) - 1 and morse_code[i+1] != ' ' and morse_code[i+1] != '/':
                silence = np.zeros(int(element_gap * sample_rate))
                signal.extend(silence)
                
        elif char == '-':
            # Adaugam o linie (ton lung)
            t = np.linspace(0, dash_duration, int(dash_duration * sample_rate), False)
            tone = np.sin(2 * np.pi * frequency * t)
            signal.extend(tone)
            
            # Adaugam o pauza dupa element (exceptand ultimul)
            if i < len(morse_code) - 1 and morse_code[i+1] != ' ' and morse_code[i+1] != '/':
                silence = np.zeros(int(element_gap * sample_rate))
                signal.extend(silence)
                
        elif char == ' ':
            # Adaugam pauza intre caractere
            silence = np.zeros(int(char_gap * sample_rate))
            signal.extend(silence)
            
        elif char == '/':
            # Adaugam pauza lunga intre cuvinte
            silence = np.zeros(int(7 * dot_duration * sample_rate))
            signal.extend(silence)
    
    # Convertim la array numpy
    signal = np.array(signal)
    
    # Normalizam intre -32767 si 32767 (range pentru int16)
    if signal.size > 0:  # Verifica daca array-ul nu este gol
        signal = signal * 32767 / max(abs(signal))
    
    # Salvam ca WAV
    wav.write(output_file, sample_rate, signal.astype(np.int16))
    
    print(f"Fisier audio generat: {output_file}")
    return morse_code

# Exemplu de utilizare:
# morse_to_audio("SOS", "morse_sos.wav")
# morse_to_audio("HELLO WORLD", "morse_hello.wav")

In [26]:
morse_to_audio("SOS", "morse_sos.wav")
morse_to_audio("HELLO WORLD", "morse_hello.wav")

Textul 'SOS' a fost convertit in codul Morse: None


TypeError: 'NoneType' object is not iterable

### 11. Initializare și rulare aplicatie

Aceasta sectiune initializeaza aplicatia si porneste interfata grafica.

In [27]:
def main():
    global app    
    app = QApplication(sys.argv) 
    init_ui()
    window.show()
    sys.exit(app.exec_())
    
if __name__ == "__main__":
    main()

Verificare initializare variabile importante:
spectrogram_canvas initializat: True
frequency_spectrum_canvas initializat: True
current_audio_file initializat: False


AttributeError: module 'scipy.signal' has no attribute 'dtype'

SystemExit: 0

e:\Anaconda\envs\am\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Concluzie si Bibliografie
Acesta este un proiect interactiv pentru redarea si manipularea fisierelor audio. Functionalitatile pot fi extinse prin implementarea exercitiilor propuse sau prin adaugarea de noi efecte si vizualizari. Proiectul ofera o baza solida pentru intelegerea procesarii audio digitale.
Bibliografie:

* PyQt5 Documentation: https://riverbankcomputing.com/software/pyqt/intro
* sounddevice Documentation: https://python-sounddevice.readthedocs.io/en/0.4.4/
* NumPy Documentation: https://numpy.org/doc/stable/
* Wave File Format: https://en.wikipedia.org/wiki/WAV
* Sound Waves Documentation: https://www.ams.jhu.edu/dan-mathofmusic/sound-waves/
* Wav vs Mp3 : https://www.gumlet.com/learn/wav-vs-mp3/
* Librosa: http://github.com/librosa/librosa
* SciPy Signal Processing: https://docs.scipy.org/doc/scipy/reference/signal.html
* Audio Signal Processing: https://en.wikipedia.org/wiki/Audio_signal_processing